In [ ]:
# %% [markdown]
# # CyberGuard Threat Analysis Notebook
# 
# ## Overview
# This notebook demonstrates CyberGuard's threat analysis capabilities including:
# 1. Web Security Scanning
# 2. Vulnerability Detection
# 3. Threat Intelligence Correlation
# 4. Risk Assessment and Scoring
# 5. Automated Response Generation
# 
# ## Key Features:
# - OWASP Top-10 vulnerability detection
# - Real-time threat intelligence integration
# - Multi-agent threat correlation
# - Automated risk scoring
# - Actionable remediation recommendations

# %% [markdown]
# ## 1. Setup and Imports

# %%
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Any, Optional, Tuple
import json
import yaml
import requests
from datetime import datetime, timedelta
import hashlib
import ipaddress
import urllib.parse
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Import CyberGuard modules
import sys
sys.path.append('..')

from src.web_security.scanner import WebSecurityScanner
from src.agents.threat_detection_agent import WebThreatDetectionAgent
from src.agents.traffic_anomaly_agent import TrafficAnomalyAgent
from src.agents.bot_detection_agent import BotDetectionAgent
from src.agents.agent_orchestrator import AgentOrchestrator
from src.data_ingestion.threat_feeds import ThreatIntelligenceIngestor
from src.utils.security_utils import validate_url, sanitize_input, calculate_risk_score

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

print("✅ CyberGuard Threat Analysis Environment Ready")
print(f"📅 Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# %% [markdown]
# ## 2. Web Security Scanner Initialization

# %%
def initialize_security_scanner(config_path: str = "../config/enterprise_config.yaml") -> WebSecurityScanner:
    """
    Initialize the web security scanner with enterprise configuration.
    
    Configuration includes:
    - Scan depth limits
    - Timeout settings
    - Security header checks
    - Vulnerability patterns
    - Threat intelligence sources
    """
    
    # Load configuration
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)
    
    # Extract security-specific config
    security_config = config.get('security', {})
    scanner_config = {
        'max_scan_depth': security_config.get('max_scan_depth', 3),
        'scan_timeout': security_config.get('scan_timeout', 30),
        'request_timeout': security_config.get('request_timeout', 10),
        'user_agent': security_config.get('user_agent', 'CyberGuard-Security-Scanner/1.0'),
        'enable_javascript_analysis': security_config.get('enable_javascript_analysis', True),
        'enable_form_analysis': security_config.get('enable_form_analysis', True),
        'enable_api_discovery': security_config.get('enable_api_discovery', True),
        'follow_redirects': security_config.get('follow_redirects', True),
        'verify_ssl': security_config.get('verify_ssl', True)
    }
    
    # Initialize scanner
    scanner = WebSecurityScanner(scanner_config)
    
    print("✅ Web Security Scanner Initialized")
    print(f"   • Max Scan Depth: {scanner_config['max_scan_depth']}")
    print(f"   • Timeout: {scanner_config['scan_timeout']}s")
    print(f"   • JavaScript Analysis: {scanner_config['enable_javascript_analysis']}")
    print(f"   • Form Analysis: {scanner_config['enable_form_analysis']}")
    
    return scanner

# Initialize scanner
scanner = initialize_security_scanner()

# %% [markdown]
# ## 3. Threat Intelligence Integration

# %%
def initialize_threat_intelligence() -> ThreatIntelligenceIngestor:
    """
    Initialize threat intelligence feeds for real-time threat data.
    
    Sources include:
    - CVE databases (MITRE, NVD)
    - Exploit databases (ExploitDB)
    - Malware feeds (VirusTotal, AbuseIPDB)
    - Phishing feeds
    - Botnet IP lists
    """
    
    threat_feeds_config = {
        'cve_feeds': [
            'https://cve.mitre.org/data/downloads/allitems.csv',
            'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-recent.json.gz'
        ],
        'exploit_feeds': [
            'https://raw.githubusercontent.com/vulnersCom/vulners-whitelist/master/exploitdb.csv'
        ],
        'malware_feeds': [
            'https://feodotracker.abuse.ch/downloads/ipblocklist.csv',
            'https://reputation.alienvault.com/reputation.data',
            'https://rules.emergingthreats.net/blockrules/compromised-ips.txt'
        ],
        'phishing_feeds': [
            'https://openphish.com/feed.txt',
            'https://phishtank.org/developer_info.php'
        ],
        'update_frequency': 'hourly',  # How often to update feeds
        'cache_duration': 3600,  # Cache duration in seconds
        'max_cache_size': 1000000  # Maximum cache entries
    }
    
    # Initialize ingestor
    ingestor = ThreatIntelligenceIngestor(threat_feeds_config)
    
    # Load initial feeds (this might take a minute)
    print("📡 Loading threat intelligence feeds...")
    ingestor.update_feeds()
    
    print("\n✅ Threat Intelligence Initialized")
    print(f"   • CVE Records: {len(ingestor.get_cve_records())}")
    print(f"   • Malware IPs: {len(ingestor.get_malware_ips())}")
    print(f"   • Phishing URLs: {len(ingestor.get_phishing_urls())}")
    print(f"   • Exploit Records: {len(ingestor.get_exploit_records())}")
    
    return ingestor

# Initialize threat intelligence
threat_intel = initialize_threat_intelligence()

# %% [markdown]
# ## 4. Security Agent Initialization

# %%
def initialize_security_agents() -> AgentOrchestrator:
    """
    Initialize and configure all security agents.
    
    Agents include:
    1. Web Threat Detection Agent (OWASP Top-10)
    2. Traffic Anomaly Detection Agent
    3. Bot & Abuse Detection Agent
    4. Malware Payload Agent
    5. Exploit Chain Reasoning Agent
    6. Digital Forensics Agent
    7. Incident Response Agent
    8. Compliance & Privacy Agent
    9. Secure Code Review Agent
    10. Threat Education Agent
    """
    
    # Initialize orchestrator
    orchestrator = AgentOrchestrator(state_dim=512)
    
    # Initialize and register agents
    print("🤖 Initializing Security Agents...")
    
    # 1. Web Threat Detection Agent
    threat_agent = WebThreatDetectionAgent("threat_detection_001")
    orchestrator.register_agent(threat_agent)
    print("   • Web Threat Detection Agent: ✅")
    
    # 2. Traffic Anomaly Agent
    traffic_agent = TrafficAnomalyAgent("traffic_anomaly_001")
    orchestrator.register_agent(traffic_agent)
    print("   • Traffic Anomaly Agent: ✅")
    
    # 3. Bot Detection Agent
    bot_agent = BotDetectionAgent("bot_detection_001")
    orchestrator.register_agent(bot_agent)
    print("   • Bot Detection Agent: ✅")
    
    # Note: Other agents would be initialized similarly
    # For this demo, we'll focus on the first three
    
    print(f"\n✅ {len(orchestrator.agents)} Security Agents Initialized")
    
    return orchestrator

# Initialize agents
agent_orchestrator = initialize_security_agents()

# %% [markdown]
# ## 5. Web Security Analysis Functions

# %%
def analyze_website_security(url: str, 
                           scanner: WebSecurityScanner,
                           orchestrator: AgentOrchestrator,
                           threat_intel: ThreatIntelligenceIngestor) -> Dict[str, Any]:
    """
    Perform comprehensive security analysis of a website.
    
    Analysis includes:
    1. Technical security scanning
    2. Threat intelligence correlation
    3. Multi-agent threat assessment
    4. Risk scoring and recommendations
    """
    
    print(f"\n{'='*80}")
    print(f"🔍 SECURITY ANALYSIS: {url}")
    print(f"{'='*80}")
    
    # Step 1: Validate URL
    if not validate_url(url):
        return {
            'error': 'Invalid URL',
            'url': url,
            'timestamp': datetime.now().isoformat()
        }
    
    start_time = datetime.now()
    
    # Step 2: Perform security scan
    print("\n📊 Step 1: Performing security scan...")
    scan_results = scanner.scan_website(url)
    
    # Step 3: Check threat intelligence
    print("📡 Step 2: Checking threat intelligence...")
    threat_intel_results = check_threat_intelligence(url, threat_intel)
    
    # Step 4: Multi-agent analysis
    print("🤖 Step 3: Multi-agent threat assessment...")
    security_data = prepare_security_data(url, scan_results, threat_intel_results)
    agent_results = orchestrator.coordinate_analysis(security_data)
    
    # Step 5: Generate comprehensive report
    print("📋 Step 4: Generating security report...")
    report = generate_security_report(
        url=url,
        scan_results=scan_results,
        threat_intel_results=threat_intel_results,
        agent_results=agent_results,
        start_time=start_time
    )
    
    analysis_duration = (datetime.now() - start_time).total_seconds()
    print(f"\n⏱️  Analysis completed in {analysis_duration:.2f} seconds")
    
    return report

def check_threat_intelligence(url: str, 
                            threat_intel: ThreatIntelligenceIngestor) -> Dict[str, Any]:
    """
    Check URL against threat intelligence databases.
    """
    
    results = {
        'cve_matches': [],
        'malware_matches': [],
        'phishing_matches': [],
        'exploit_matches': [],
        'reputation_score': 0.0
    }
    
    try:
        # Parse URL
        parsed_url = urllib.parse.urlparse(url)
        domain = parsed_url.netloc
        ip_address = None
        
        # Try to resolve domain to IP
        try:
            import socket
            ip_address = socket.gethostbyname(domain)
        except:
            pass
        
        # Check CVE database for technology vulnerabilities
        # This would check server headers, technologies detected, etc.
        # For demo, we'll simulate some checks
        
        # Check malware IP databases
        if ip_address:
            malware_ips = threat_intel.get_malware_ips()
            if ip_address in malware_ips:
                results['malware_matches'].append({
                    'type': 'malware_ip',
                    'ip': ip_address,
                    'source': 'malware_feed',
                    'severity': 'HIGH'
                })
        
        # Check phishing databases
        phishing_urls = threat_intel.get_phishing_urls()
        # Simple substring check for demo
        for phishing_url in phishing_urls[:100]:  # Check first 100 for demo
            if domain in phishing_url:
                results['phishing_matches'].append({
                    'type': 'phishing_domain',
                    'domain': domain,
                    'source': 'phishing_feed',
                    'severity': 'CRITICAL'
                })
                break
        
        # Calculate reputation score
        # Higher score = worse reputation
        reputation = 0.0
        
        if results['malware_matches']:
            reputation += 0.7
        if results['phishing_matches']:
            reputation += 0.9
        
        # Check if domain is new (less than 30 days old)
        # This requires WHOIS data, simplified for demo
        domain_age = check_domain_age(domain)
        if domain_age < 30:  # Less than 30 days old
            reputation += 0.3
        
        results['reputation_score'] = min(1.0, reputation)
        
    except Exception as e:
        print(f"⚠️ Threat intelligence check error: {e}")
        results['error'] = str(e)
    
    return results

def check_domain_age(domain: str) -> int:
    """
    Check domain age in days.
    In production, this would use WHOIS data.
    For demo, return random age.
    """
    import random
    return random.randint(1, 365 * 5)  # Random age between 1 day and 5 years

def prepare_security_data(url: str, 
                         scan_results: Dict,
                         threat_intel_results: Dict) -> Dict[str, Any]:
    """
    Prepare security data for agent analysis.
    """
    
    return {
        'url': url,
        'scan_results': scan_results,
        'threat_intel': threat_intel_results,
        'timestamp': datetime.now().isoformat(),
        'metadata': {
            'scan_id': hashlib.md5(f"{url}_{datetime.now().isoformat()}".encode()).hexdigest(),
            'analysis_type': 'comprehensive_security_scan',
            'data_source': 'web_scanner_threat_intel'
        }
    }

def generate_security_report(url: str,
                           scan_results: Dict,
                           threat_intel_results: Dict,
                           agent_results: Dict,
                           start_time: datetime) -> Dict[str, Any]:
    """
    Generate comprehensive security report.
    """
    
    # Extract final decision from agent coordination
    final_decision = agent_results.get('final_decision', {})
    
    # Calculate overall risk score
    risk_score = calculate_overall_risk(
        scan_results, 
        threat_intel_results, 
        final_decision
    )
    
    # Generate recommendations
    recommendations = generate_recommendations(
        scan_results,
        threat_intel_results,
        final_decision
    )
    
    # Determine threat level category
    threat_level = final_decision.get('threat_level', 0.0)
    if threat_level >= 0.8:
        threat_category = 'CRITICAL'
        color = '#ff0000'  # Red
    elif threat_level >= 0.6:
        threat_category = 'HIGH'
        color = '#ff6600'  # Orange
    elif threat_level >= 0.4:
        threat_category = 'MEDIUM'
        color = '#ffcc00'  # Yellow
    elif threat_level >= 0.2:
        threat_category = 'LOW'
        color = '#0099ff'  # Blue
    else:
        threat_category = 'INFORMATIONAL'
        color = '#00cc66'  # Green
    
    # Compile report
    report = {
        'metadata': {
            'url': url,
            'scan_id': scan_results.get('scan_id', ''),
            'analysis_start': start_time.isoformat(),
            'analysis_end': datetime.now().isoformat(),
            'analysis_duration': (datetime.now() - start_time).total_seconds(),
            'report_version': '1.0',
            'cyberguard_version': '1.0.0'
        },
        'executive_summary': {
            'threat_level': threat_level,
            'threat_category': threat_category,
            'color': color,
            'confidence': final_decision.get('confidence', 0.0),
            'risk_score': risk_score,
            'key_findings': extract_key_findings(scan_results, threat_intel_results),
            'recommended_action': final_decision.get('action', 'MONITOR')
        },
        'technical_findings': {
            'security_scan': scan_results,
            'threat_intelligence': threat_intel_results,
            'agent_analysis': agent_results
        },
        'vulnerabilities': compile_vulnerabilities(scan_results, threat_intel_results),
        'recommendations': recommendations,
        'agent_contributions': agent_results.get('agent_contributions', []),
        'evidence': final_decision.get('evidence', [])[:10],  # Top 10 pieces of evidence
        'requires_human_review': final_decision.get('requires_human_review', False)
    }
    
    return report

def calculate_overall_risk(scan_results: Dict,
                         threat_intel_results: Dict,
                         final_decision: Dict) -> float:
    """
    Calculate overall risk score (0.0 to 1.0).
    """
    
    # Weight different components
    weights = {
        'scan_risk': 0.4,  # Technical scan findings
        'threat_intel': 0.3,  # Threat intelligence matches
        'agent_assessment': 0.3  # Multi-agent assessment
    }
    
    # 1. Scan risk
    scan_risk = scan_results.get('risk_score', 0.0)
    
    # 2. Threat intelligence risk
    threat_intel_risk = threat_intel_results.get('reputation_score', 0.0)
    
    # 3. Agent assessment risk
    agent_risk = final_decision.get('threat_level', 0.0)
    
    # Weighted average
    overall_risk = (
        scan_risk * weights['scan_risk'] +
        threat_intel_risk * weights['threat_intel'] +
        agent_risk * weights['agent_assessment']
    )
    
    return min(1.0, overall_risk)

def extract_key_findings(scan_results: Dict, 
                        threat_intel_results: Dict) -> List[str]:
    """
    Extract key findings for executive summary.
    """
    
    findings = []
    
    # From security scan
    vulnerabilities = scan_results.get('vulnerabilities', [])
    if vulnerabilities:
        critical_vulns = [v for v in vulnerabilities if v.get('severity') == 'CRITICAL']
        high_vulns = [v for v in vulnerabilities if v.get('severity') == 'HIGH']
        
        if critical_vulns:
            findings.append(f"{len(critical_vulns)} critical vulnerabilities detected")
        if high_vulns:
            findings.append(f"{len(high_vulns)} high-severity vulnerabilities detected")
    
    # Security headers
    security_headers = scan_results.get('security_headers', {})
    missing_headers = [h for h, info in security_headers.items() 
                      if not info.get('present', False)]
    if missing_headers:
        findings.append(f"{len(missing_headers)} critical security headers missing")
    
    # From threat intelligence
    if threat_intel_results.get('malware_matches'):
        findings.append("Domain associated with malware activity")
    
    if threat_intel_results.get('phishing_matches'):
        findings.append("Domain matches known phishing patterns")
    
    # If no major findings
    if not findings:
        findings.append("No critical security issues detected")
    
    return findings[:5]  # Limit to 5 key findings

def compile_vulnerabilities(scan_results: Dict,
                          threat_intel_results: Dict) -> List[Dict]:
    """
    Compile all vulnerabilities from different sources.
    """
    
    vulnerabilities = []
    
    # Add scan vulnerabilities
    scan_vulns = scan_results.get('vulnerabilities', [])
    for vuln in scan_vulns:
        vuln['source'] = 'security_scan'
        vulnerabilities.append(vuln)
    
    # Add threat intelligence vulnerabilities
    for match_type in ['malware_matches', 'phishing_matches', 'exploit_matches']:
        matches = threat_intel_results.get(match_type, [])
        for match in matches:
            match['source'] = 'threat_intelligence'
            vulnerabilities.append(match)
    
    # Sort by severity
    severity_order = {'CRITICAL': 0, 'HIGH': 1, 'MEDIUM': 2, 'LOW': 3, 'INFORMATIONAL': 4}
    vulnerabilities.sort(key=lambda x: severity_order.get(x.get('severity', 'INFORMATIONAL'), 4))
    
    return vulnerabilities

def generate_recommendations(scan_results: Dict,
                           threat_intel_results: Dict,
                           final_decision: Dict) -> List[Dict]:
    """
    Generate actionable security recommendations.
    """
    
    recommendations = []
    
    # 1. Recommendations from security scan
    scan_recommendations = scan_results.get('recommendations', [])
    for rec in scan_recommendations[:5]:  # Top 5
        recommendations.append({
            'type': 'technical',
            'priority': 'HIGH' if any(word in rec.lower() for word in ['critical', 'immediate', 'urgent']) else 'MEDIUM',
            'recommendation': rec,
            'category': 'security_configuration'
        })
    
    # 2. Recommendations from threat intelligence
    if threat_intel_results.get('malware_matches'):
        recommendations.append({
            'type': 'threat_intel',
            'priority': 'CRITICAL',
            'recommendation': 'Block traffic from this domain due to malware associations',
            'category': 'network_security'
        })
    
    if threat_intel_results.get('phishing_matches'):
        recommendations.append({
            'type': 'threat_intel',
            'priority': 'CRITICAL',
            'recommendation': 'Add domain to phishing blocklist and warn users',
            'category': 'user_security'
        })
    
    # 3. Recommendations from agent decision
    agent_action = final_decision.get('action', '')
    if agent_action == 'BLOCK':
        recommendations.append({
            'type': 'agent_decision',
            'priority': 'CRITICAL',
            'recommendation': 'Immediately block all traffic to/from this domain',
            'category': 'incident_response'
        })
    elif agent_action == 'CHALLENGE':
        recommendations.append({
            'type': 'agent_decision',
            'priority': 'HIGH',
            'recommendation': 'Implement additional verification (CAPTCHA, 2FA) for requests',
            'category': 'access_control'
        })
    
    # 4. General recommendations based on risk
    risk_score = calculate_overall_risk(scan_results, threat_intel_results, final_decision)
    if risk_score > 0.7:
        recommendations.extend([
            {
                'type': 'general',
                'priority': 'HIGH',
                'recommendation': 'Conduct immediate security audit and penetration testing',
                'category': 'security_governance'
            },
            {
                'type': 'general',
                'priority': 'HIGH',
                'recommendation': 'Implement Web Application Firewall (WAF) with strict rules',
                'category': 'infrastructure_security'
            }
        ])
    elif risk_score > 0.4:
        recommendations.append({
            'type': 'general',
            'priority': 'MEDIUM',
            'recommendation': 'Schedule security review and implement monitoring',
            'category': 'security_operations'
        })
    
    # Remove duplicates
    unique_recommendations = []
    seen = set()
    for rec in recommendations:
        rec_key = rec['recommendation'][:50]  # First 50 chars as key
        if rec_key not in seen:
            seen.add(rec_key)
            unique_recommendations.append(rec)
    
    return unique_recommendations[:10]  # Limit to 10 recommendations

# %% [markdown]
# ## 6. Website Security Analysis Examples

# %%
# Example 1: Analyze a test website
print("🧪 Example 1: Analyzing Test Website")
print("="*60)

test_url = "https://httpbin.org"  # A safe test website
report = analyze_website_security(
    url=test_url,
    scanner=scanner,
    orchestrator=agent_orchestrator,
    threat_intel=threat_intel
)

# Display summary
print(f"\n📋 SECURITY REPORT SUMMARY - {test_url}")
print(f"{'='*60}")
print(f"Threat Level: {report['executive_summary']['threat_category']}")
print(f"Risk Score: {report['executive_summary']['risk_score']:.2f}")
print(f"Confidence: {report['executive_summary']['confidence']:.2f}")
print(f"Recommended Action: {report['executive_summary']['recommended_action']}")

print(f"\n🔑 Key Findings:")
for finding in report['executive_summary']['key_findings']:
    print(f"  • {finding}")

print(f"\n💡 Top Recommendations:")
for i, rec in enumerate(report['recommendations'][:3], 1):
    print(f"  {i}. [{rec['priority']}] {rec['recommendation']}")

# %% [markdown]
# ## 7. Vulnerability Analysis Dashboard

# %%
def create_vulnerability_dashboard(report: Dict) -> plt.Figure:
    """
    Create comprehensive vulnerability dashboard visualization.
    """
    
    fig = plt.figure(figsize=(16, 12))
    
    # Create gridspec for complex layout
    gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
    
    # 1. Threat Level Gauge (Top-left)
    ax1 = fig.add_subplot(gs[0, 0])
    plot_threat_gauge(ax1, report['executive_summary']['threat_level'])
    
    # 2. Risk Breakdown (Top-center)
    ax2 = fig.add_subplot(gs[0, 1])
    plot_risk_breakdown(ax2, report)
    
    # 3. Vulnerability Distribution (Top-right)
    ax3 = fig.add_subplot(gs[0, 2])
    plot_vulnerability_distribution(ax3, report['vulnerabilities'])
    
    # 4. Security Headers Analysis (Middle-left)
    ax4 = fig.add_subplot(gs[1, 0])
    plot_security_headers(ax4, report['technical_findings']['security_scan'])
    
    # 5. Agent Contribution (Middle-center)
    ax5 = fig.add_subplot(gs[1, 1])
    plot_agent_contributions(ax5, report['agent_contributions'])
    
    # 6. Threat Intelligence (Middle-right)
    ax6 = fig.add_subplot(gs[1, 2])
    plot_threat_intelligence(ax6, report['technical_findings']['threat_intelligence'])
    
    # 7. Recommendation Priority (Bottom-left)
    ax7 = fig.add_subplot(gs[2, 0])
    plot_recommendation_priority(ax7, report['recommendations'])
    
    # 8. Timeline Analysis (Bottom-center)
    ax8 = fig.add_subplot(gs[2, 1])
    plot_timeline_analysis(ax8, report)
    
    # 9. Confidence Metrics (Bottom-right)
    ax9 = fig.add_subplot(gs[2, 2])
    plot_confidence_metrics(ax9, report)
    
    # Add title
    fig.suptitle(
        f"CyberGuard Security Dashboard - {report['metadata']['url']}\n"
        f"Analysis: {report['metadata']['analysis_start'][:10]} | "
        f"Duration: {report['metadata']['analysis_duration']:.1f}s",
        fontsize=14, fontweight='bold'
    )
    
    return fig

def plot_threat_gauge(ax, threat_level: float):
    """Plot threat level gauge."""
    
    # Create gauge
    theta = np.linspace(0, np.pi, 100)
    r = np.ones_like(theta)
    
    # Threat level angle
    threat_angle = threat_level * np.pi
    
    # Plot gauge background
    ax.plot(theta, r, 'k-', linewidth=3)
    
    # Color sections
    sections = [
        (0, 0.2, 'green', 'LOW'),
        (0.2, 0.4, 'lightgreen', 'GUARDED'),
        (0.4, 0.6, 'yellow', 'ELEVATED'),
        (0.6, 0.8, 'orange', 'HIGH'),
        (0.8, 1.0, 'red', 'CRITICAL')
    ]
    
    for start, end, color, label in sections:
        section_theta = np.linspace(start * np.pi, end * np.pi, 20)
        section_r = np.ones_like(section_theta)
        ax.fill_between(section_theta, 0, section_r, alpha=0.3, color=color)
        
        # Add label at middle of section
        mid_angle = (start + end) * np.pi / 2
        ax.text(mid_angle, 1.1, label, ha='center', va='center', 
               fontsize=9, fontweight='bold', rotation=mid_angle*180/np.pi - 90)
    
    # Plot threat needle
    needle_length = 0.9
    ax.plot([threat_angle, threat_angle], [0, needle_length], 
           'r-', linewidth=3, alpha=0.8)
    ax.plot(threat_angle, needle_length, 'ro', markersize=10)
    
    # Add threat level text
    ax.text(np.pi/2, 0.5, f'Threat Level\n{threat_level:.2f}', 
           ha='center', va='center', fontsize=12, fontweight='bold')
    
    ax.set_xlim(0, np.pi)
    ax.set_ylim(0, 1.2)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title('Threat Level Gauge', fontsize=11, fontweight='bold')

def plot_risk_breakdown(ax, report: Dict):
    """Plot risk score breakdown by component."""
    
    components = ['Scan Risk', 'Threat Intel', 'Agent Assessment']
    weights = [0.4, 0.3, 0.3]
    
    # Calculate component scores
    scan_risk = report['technical_findings']['security_scan'].get('risk_score', 0)
    threat_intel_risk = report['technical_findings']['threat_intelligence'].get('reputation_score', 0)
    agent_risk = report['executive_summary'].get('threat_level', 0)
    
    scores = [scan_risk, threat_intel_risk, agent_risk]
    weighted_scores = [s * w for s, w in zip(scores, weights)]
    
    # Create bar plot
    x = np.arange(len(components))
    width = 0.35
    
    bars1 = ax.bar(x - width/2, scores, width, label='Raw Score', alpha=0.7)
    bars2 = ax.bar(x + width/2, weighted_scores, width, label='Weighted', alpha=0.7)
    
    # Add value labels
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.2f}', ha='center', va='bottom', fontsize=8)
    
    ax.set_xlabel('Risk Component')
    ax.set_ylabel('Score')
    ax.set_title('Risk Score Breakdown')
    ax.set_xticks(x)
    ax.set_xticklabels(components, rotation=45, ha='right')
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    
    # Add overall risk
    overall_risk = report['executive_summary']['risk_score']
    ax.axhline(y=overall_risk, color='red', linestyle='--', alpha=0.7,
              label=f'Overall: {overall_risk:.2f}')
    ax.legend()

def plot_vulnerability_distribution(ax, vulnerabilities: List[Dict]):
    """Plot vulnerability distribution by severity and source."""
    
    if not vulnerabilities:
        ax.text(0.5, 0.5, 'No Vulnerabilities\nDetected', 
               ha='center', va='center', fontsize=12)
        ax.set_title('Vulnerability Distribution')
        return
    
    # Extract data
    severities = [v.get('severity', 'UNKNOWN') for v in vulnerabilities]
    sources = [v.get('source', 'unknown') for v in vulnerabilities]
    
    # Create DataFrames for easier analysis
    import pandas as pd
    df = pd.DataFrame({
        'severity': severities,
        'source': sources
    })
    
    # Plot 1: Severity distribution (pie chart)
    severity_counts = df['severity'].value_counts()
    
    # Order by severity
    severity_order = ['CRITICAL', 'HIGH', 'MEDIUM', 'LOW', 'INFORMATIONAL', 'UNKNOWN']
    severity_counts = severity_counts.reindex(severity_order, fill_value=0)
    
    colors = ['#ff0000', '#ff6600', '#ffcc00', '#0099ff', '#00cc66', '#cccccc']
    
    wedges, texts, autotexts = ax.pie(
        severity_counts.values,
        labels=severity_counts.index,
        colors=colors[:len(severity_counts)],
        autopct='%1.1f%%',
        startangle=90,
        textprops={'fontsize': 9}
    )
    
    # Enhance autotexts
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')
    
    ax.set_title('Vulnerability Severity Distribution', fontsize=11, fontweight='bold')
    
    # Add source distribution as donut hole
    if len(set(sources)) > 1:
        # Create inner pie for sources
        source_counts = df['source'].value_counts()
        
        # Create nested pie
        center_circle = plt.Circle((0, 0), 0.6, color='white', ec='black')
        ax.add_patch(center_circle)
        
        # Add source text
        source_text = "\n".join([f"{s}: {c}" for s, c in source_counts.items()])
        ax.text(0, 0, f"Sources:\n{source_text}", 
               ha='center', va='center', fontsize=8)

def plot_security_headers(ax, scan_results: Dict):
    """Plot security headers analysis."""
    
    headers = scan_results.get('security_headers', {})
    
    if not headers:
        ax.text(0.5, 0.5, 'No Header Data', 
               ha='center', va='center', fontsize=12)
        ax.set_title('Security Headers')
        return
    
    # Prepare data
    header_names = []
    present = []
    secure = []
    
    for header, info in headers.items():
        header_names.append(header)
        present.append(1 if info.get('present', False) else 0)
        secure.append(1 if info.get('secure', False) else 0)
    
    # Create stacked bar chart
    x = np.arange(len(header_names))
    width = 0.8
    
    bars1 = ax.bar(x, present, width, label='Present', color='lightblue', alpha=0.7)
    bars2 = ax.bar(x, secure, width, label='Secure', color='lightgreen', alpha=0.7, 
                  bottom=present)
    
    # Customize
    ax.set_xlabel('Security Header')
    ax.set_ylabel('Status')
    ax.set_title('Security Headers Analysis')
    ax.set_xticks(x)
    ax.set_xticklabels([h.replace('-', '\n') for h in header_names], 
                      rotation=45, ha='right', fontsize=8)
    ax.set_ylim(0, 2)
    ax.set_yticks([0.5, 1.5])
    ax.set_yticklabels(['Missing', 'Present'])
    ax.legend(['Missing', 'Present', 'Insecure', 'Secure'], 
             loc='upper right', fontsize=8)
    ax.grid(True, alpha=0.3, axis='y')

def plot_agent_contributions(ax, contributions: List):
    """Plot agent contribution weights."""
    
    if not contributions:
        ax.text(0.5, 0.5, 'No Agent Data', 
               ha='center', va='center', fontsize=12)
        ax.set_title('Agent Contributions')
        return
    
    # Flatten contributions if nested
    if isinstance(contributions[0], list):
        contributions = contributions[0]  # Take first batch
    
    # Agent names
    agent_names = [f'Agent {i+1}' for i in range(len(contributions))]
    
    # Create horizontal bar chart
    y_pos = np.arange(len(agent_names))
    
    bars = ax.barh(y_pos, contributions, color=plt.cm.Set1(np.linspace(0, 1, len(contributions))))
    
    # Add value labels
    for i, (bar, val) in enumerate(zip(bars, contributions)):
        width = bar.get_width()
        ax.text(width + 0.01, bar.get_y() + bar.get_height()/2,
               f'{val:.3f}', ha='left', va='center', fontsize=9)
    
    ax.set_yticks(y_pos)
    ax.set_yticklabels(agent_names)
    ax.set_xlabel('Contribution Weight')
    ax.set_title('Agent Contribution Distribution')
    ax.set_xlim(0, 1)
    ax.grid(True, alpha=0.3, axis='x')
    
    # Add horizontal line at equal contribution
    equal_weight = 1.0 / len(contributions)
    ax.axvline(x=equal_weight, color='red', linestyle='--', alpha=0.5,
              label=f'Equal: {equal_weight:.3f}')
    ax.legend()

def plot_threat_intelligence(ax, threat_intel: Dict):
    """Plot threat intelligence findings."""
    
    # Extract data
    malware_matches = len(threat_intel.get('malware_matches', []))
    phishing_matches = len(threat_intel.get('phishing_matches', []))
    exploit_matches = len(threat_intel.get('exploit_matches', []))
    cve_matches = len(threat_intel.get('cve_matches', []))
    reputation = threat_intel.get('reputation_score', 0)
    
    # Create radar chart
    categories = ['Malware', 'Phishing', 'Exploits', 'CVEs', 'Reputation']
    N = len(categories)
    
    # Normalize values to 0-1 scale
    values = [
        min(1.0, malware_matches / 10),  # Cap at 10 matches
        min(1.0, phishing_matches / 5),  # Cap at 5 matches
        min(1.0, exploit_matches / 5),   # Cap at 5 matches
        min(1.0, cve_matches / 20),      # Cap at 20 matches
        reputation  # Already 0-1
    ]
    
    # Repeat first value to close the polygon
    values += values[:1]
    
    # Calculate angles
    angles = np.linspace(0, 2 * np.pi, N, endpoint=False).tolist()
    angles += angles[:1]
    
    # Plot
    ax = plt.subplot(projection='polar')
    ax.plot(angles, values, 'o-', linewidth=2, color='red', alpha=0.7)
    ax.fill(angles, values, color='red', alpha=0.25)
    
    # Set category labels
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories, fontsize=9)
    
    # Set radial labels
    ax.set_rlabel_position(0)
    ax.set_yticks([0.25, 0.5, 0.75, 1.0])
    ax.set_yticklabels(['0.25', '0.5', '0.75', '1.0'], fontsize=8)
    ax.set_ylim(0, 1)
    
    ax.set_title('Threat Intelligence Profile', fontsize=11, fontweight='bold', pad=20)
    
    # Add value labels
    for angle, value, category in zip(angles[:-1], values[:-1], categories):
        # Convert to degrees
        angle_deg = angle * 180 / np.pi
        
        # Position text
        if angle_deg < 90 or angle_deg > 270:
            ha = 'left'
        else:
            ha = 'right'
        
        # Offset for better visibility
        offset = 0.1
        ax.text(angle, value + offset, f'{value:.2f}', 
               ha='center', va='center', fontsize=8, fontweight='bold')

def plot_recommendation_priority(ax, recommendations: List[Dict]):
    """Plot recommendation priority distribution."""
    
    if not recommendations:
        ax.text(0.5, 0.5, 'No Recommendations', 
               ha='center', va='center', fontsize=12)
        ax.set_title('Recommendations')
        return
    
    # Count by priority
    priorities = ['CRITICAL', 'HIGH', 'MEDIUM', 'LOW']
    counts = {p: 0 for p in priorities}
    
    for rec in recommendations:
        priority = rec.get('priority', 'MEDIUM')
        if priority in counts:
            counts[priority] += 1
    
    # Create bar chart
    x = np.arange(len(priorities))
    values = [counts[p] for p in priorities]
    colors = ['red', 'orange', 'yellow', 'green']
    
    bars = ax.bar(x, values, color=colors, alpha=0.7)
    
    # Add value labels
    for bar, val in zip(bars, values):
        height = bar.get_height()
        if height > 0:
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{val}', ha='center', va='bottom', fontsize=9, fontweight='bold')
    
    ax.set_xlabel('Priority Level')
    ax.set_ylabel('Number of Recommendations')
    ax.set_title('Recommendation Priority Distribution')
    ax.set_xticks(x)
    ax.set_xticklabels(priorities, rotation=45, ha='right')
    ax.grid(True, alpha=0.3, axis='y')
    
    # Add category breakdown
    categories = set([r.get('category', 'other') for r in recommendations])
    if len(categories) > 1:
        # Create stacked version in background (transparent)
        bottom = np.zeros(len(priorities))
        for category in categories:
            cat_counts = [sum(1 for r in recommendations 
                            if r.get('priority') == p and r.get('category') == category)
                         for p in priorities]
            if sum(cat_counts) > 0:
                ax.bar(x, cat_counts, bottom=bottom, alpha=0.3)
                bottom += cat_counts

def plot_timeline_analysis(ax, report: Dict):
    """Plot analysis timeline and component durations."""
    
    # Simulated timeline data (in production, this would come from actual timing)
    components = ['URL Validation', 'Security Scan', 'Threat Intel', 
                 'Agent Analysis', 'Report Generation']
    durations = [0.1, 15.2, 2.3, 1.8, 0.5]  # Example durations in seconds
    
    # Create timeline
    start_times = [0]
    for i in range(1, len(durations)):
        start_times.append(start_times[-1] + durations[i-1])
    
    # Create horizontal bars
    y_pos = np.arange(len(components))
    
    bars = ax.barh(y_pos, durations, left=start_times, 
                  color=plt.cm.viridis(np.linspace(0, 1, len(components))))
    
    # Add duration labels
    for i, (start, duration) in enumerate(zip(start_times, durations)):
        ax.text(start + duration/2, i, f'{duration:.1f}s', 
               ha='center', va='center', fontsize=9, fontweight='bold',
               color='white' if duration > max(durations)/2 else 'black')
    
    ax.set_yticks(y_pos)
    ax.set_yticklabels(components)
    ax.set_xlabel('Time (seconds)')
    ax.set_title('Analysis Timeline')
    ax.grid(True, alpha=0.3, axis='x')
    
    # Add total time
    total_time = sum(durations)
    ax.axvline(x=total_time, color='red', linestyle='--', alpha=0.7)
    ax.text(total_time, len(components)-0.5, f'Total: {total_time:.1f}s',
           ha='right', va='center', fontsize=10, fontweight='bold', color='red')

def plot_confidence_metrics(ax, report: Dict):
    """Plot confidence metrics and reliability indicators."""
    
    # Extract confidence data
    agent_confidence = report['executive_summary'].get('confidence', 0.5)
    scan_completeness = min(1.0, len(report['vulnerabilities']) / 20)  # Example metric
    data_freshness = 0.9  # Example: threat intel is 90% fresh
    agent_agreement = 0.8  # Example: agents agree 80%
    
    metrics = {
        'Agent Confidence': agent_confidence,
        'Scan Completeness': scan_completeness,
        'Data Freshness': data_freshness,
        'Agent Agreement': agent_agreement
    }
    
    # Calculate overall reliability
    weights = [0.4, 0.2, 0.2, 0.2]  # Weight for each metric
    overall_reliability = sum(v * w for v, w in zip(metrics.values(), weights))
    
    # Create gauge for overall reliability
    # Inspired by matplotlib gauge example
    theta = np.linspace(0, np.pi, 100)
    r = np.ones_like(theta)
    
    # Plot gauge background
    ax.plot(theta, r, 'k-', linewidth=2)
    
    # Color sections for reliability
    reliability_colors = [
        (0, 0.6, 'red', 'Low'),
        (0.6, 0.8, 'yellow', 'Medium'),
        (0.8, 1.0, 'green', 'High')
    ]
    
    for start, end, color, label in reliability_colors:
        section_theta = np.linspace(start * np.pi, end * np.pi, 20)
        section_r = np.ones_like(section_theta)
        ax.fill_between(section_theta, 0, section_r, alpha=0.3, color=color)
    
    # Plot reliability needle
    reliability_angle = overall_reliability * np.pi
    ax.plot([reliability_angle, reliability_angle], [0, 0.9], 
           'b-', linewidth=3, alpha=0.8)
    ax.plot(reliability_angle, 0.9, 'bo', markersize=10)
    
    # Add reliability value
    ax.text(np.pi/2, 0.5, f'Reliability\n{overall_reliability:.2f}', 
           ha='center', va='center', fontsize=12, fontweight='bold')
    
    ax.set_xlim(0, np.pi)
    ax.set_ylim(0, 1.2)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title('Analysis Reliability', fontsize=11, fontweight='bold')
    
    # Add metrics as text below gauge
    metrics_text = "\n".join([f"{k}: {v:.2f}" for k, v in metrics.items()])
    ax.text(np.pi/2, -0.2, metrics_text, ha='center', va='top', 
           fontsize=9, bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.5))

# Create dashboard for the test website
print("\n📊 Creating Vulnerability Dashboard...")
dashboard_fig = create_vulnerability_dashboard(report)
plt.tight_layout()
plt.show()

# %% [markdown]
# ## 8. Batch Analysis and Comparative Reports

# %%
def analyze_multiple_websites(urls: List[str],
                            scanner: WebSecurityScanner,
                            orchestrator: AgentOrchestrator,
                            threat_intel: ThreatIntelligenceIngestor) -> pd.DataFrame:
    """
    Analyze multiple websites and create comparative report.
    """
    
    print(f"\n📊 BATCH ANALYSIS: {len(urls)} Websites")
    print("="*60)
    
    all_reports = []
    
    for i, url in enumerate(tqdm(urls, desc="Analyzing websites")):
        try:
            print(f"\n[{i+1}/{len(urls)}] Analyzing: {url}")
            
            report = analyze_website_security(
                url=url,
                scanner=scanner,
                orchestrator=orchestrator,
                threat_intel=threat_intel
            )
            
            # Extract key metrics for comparison
            summary = {
                'url': url,
                'threat_level': report['executive_summary']['threat_level'],
                'threat_category': report['executive_summary']['threat_category'],
                'risk_score': report['executive_summary']['risk_score'],
                'confidence': report['executive_summary']['confidence'],
                'recommended_action': report['executive_summary']['recommended_action'],
                'vulnerability_count': len(report['vulnerabilities']),
                'critical_vulns': len([v for v in report['vulnerabilities'] 
                                      if v.get('severity') == 'CRITICAL']),
                'high_vulns': len([v for v in report['vulnerabilities'] 
                                  if v.get('severity') == 'HIGH']),
                'missing_headers': sum(1 for h in report['technical_findings']
                                      ['security_scan'].get('security_headers', {}).values()
                                      if not h.get('present', False)),
                'analysis_duration': report['metadata']['analysis_duration'],
                'requires_human_review': report['requires_human_review']
            }
            
            all_reports.append(summary)
            
        except Exception as e:
            print(f"⚠️ Error analyzing {url}: {e}")
            all_reports.append({
                'url': url,
                'error': str(e),
                'threat_level': 0.5,
                'threat_category': 'UNKNOWN',
                'risk_score': 0.5
            })
    
    # Create DataFrame
    df = pd.DataFrame(all_reports)
    
    # Calculate statistics
    print(f"\n{'='*60}")
    print("BATCH ANALYSIS SUMMARY")
    print(f"{'='*60}")
    
    if not df.empty:
        print(f"Total Websites Analyzed: {len(df)}")
        print(f"Average Threat Level: {df['threat_level'].mean():.3f}")
        print(f"Average Risk Score: {df['risk_score'].mean():.3f}")
        print(f"Total Critical Vulnerabilities: {df['critical_vulns'].sum()}")
        print(f"Total High Vulnerabilities: {df['high_vulns'].sum()}")
        print(f"Websites Requiring Human Review: {df['requires_human_review'].sum()}")
        
        # Distribution of recommended actions
        action_counts = df['recommended_action'].value_counts()
        print("\nRecommended Actions Distribution:")
        for action, count in action_counts.items():
            print(f"  {action}: {count} websites")
    
    return df

def create_comparative_analysis(df: pd.DataFrame) -> plt.Figure:
    """
    Create comparative analysis visualization for multiple websites.
    """
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    # 1. Threat Level Distribution
    ax1 = axes[0, 0]
    if 'threat_level' in df.columns:
        ax1.hist(df['threat_level'], bins=20, alpha=0.7, color='red', edgecolor='black')
        ax1.set_xlabel('Threat Level')
        ax1.set_ylabel('Frequency')
        ax1.set_title('Threat Level Distribution')
        ax1.grid(True, alpha=0.3)
        ax1.axvline(x=df['threat_level'].mean(), color='blue', linestyle='--',
                   label=f'Mean: {df["threat_level"].mean():.3f}')
        ax1.legend()
    
    # 2. Risk Score vs Threat Level
    ax2 = axes[0, 1]
    if 'risk_score' in df.columns and 'threat_level' in df.columns:
        scatter = ax2.scatter(df['risk_score'], df['threat_level'], 
                             c=df.get('critical_vulns', 0), 
                             cmap='Reds', s=100, alpha=0.7)
        ax2.set_xlabel('Risk Score')
        ax2.set_ylabel('Threat Level')
        ax2.set_title('Risk vs Threat Correlation')
        ax2.grid(True, alpha=0.3)
        
        # Add colorbar for critical vulnerabilities
        plt.colorbar(scatter, ax=ax2, label='Critical Vulnerabilities')
        
        # Add trend line
        if len(df) > 1:
            z = np.polyfit(df['risk_score'], df['threat_level'], 1)
            p = np.poly1d(z)
            ax2.plot(sorted(df['risk_score']), p(sorted(df['risk_score'])), 
                    "r--", alpha=0.5, label='Trend')
            ax2.legend()
    
    # 3. Vulnerability Distribution
    ax3 = axes[0, 2]
    if 'critical_vulns' in df.columns and 'high_vulns' in df.columns:
        categories = ['Critical', 'High']
        avg_counts = [df['critical_vulns'].mean(), df['high_vulns'].mean()]
        
        bars = ax3.bar(categories, avg_counts, color=['red', 'orange'], alpha=0.7)
        ax3.set_xlabel('Severity')
        ax3.set_ylabel('Average Count')
        ax3.set_title('Average Vulnerabilities per Website')
        ax3.grid(True, alpha=0.3, axis='y')
        
        # Add value labels
        for bar, count in zip(bars, avg_counts):
            height = bar.get_height()
            ax3.text(bar.get_x() + bar.get_width()/2., height,
                    f'{count:.1f}', ha='center', va='bottom', fontsize=10)
    
    # 4. Recommended Actions Distribution
    ax4 = axes[1, 0]
    if 'recommended_action' in df.columns:
        action_counts = df['recommended_action'].value_counts()
        
        # Create donut chart
        wedges, texts, autotexts = ax4.pie(
            action_counts.values,
            labels=action_counts.index,
            autopct='%1.1f%%',
            startangle=90,
            pctdistance=0.85,
            colors=plt.cm.Set2(np.linspace(0, 1, len(action_counts)))
        )
        
        # Draw circle for donut effect
        centre_circle = plt.Circle((0, 0), 0.70, fc='white')
        ax4.add_artist(centre_circle)
        
        ax4.set_title('Recommended Actions Distribution')
        
        # Enhance autotexts
        for autotext in autotexts:
            autotext.set_color('black')
            autotext.set_fontweight('bold')
    
    # 5. Analysis Duration Distribution
    ax5 = axes[1, 1]
    if 'analysis_duration' in df.columns:
        ax5.boxplot(df['analysis_duration'], vert=False)
        ax5.scatter(df['analysis_duration'], np.ones(len(df)), 
                   alpha=0.5, color='red', s=50)
        ax5.set_xlabel('Analysis Duration (seconds)')
        ax5.set_title('Analysis Performance')
        ax5.grid(True, alpha=0.3, axis='x')
        ax5.set_yticks([])
        
        # Add statistics
        stats_text = f"Mean: {df['analysis_duration'].mean():.1f}s\n"
        stats_text += f"Std: {df['analysis_duration'].std():.1f}s\n"
        stats_text += f"Min: {df['analysis_duration'].min():.1f}s\n"
        stats_text += f"Max: {df['analysis_duration'].max():.1f}s"
        
        ax5.text(0.05, 0.95, stats_text, transform=ax5.transAxes,
                fontsize=9, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # 6. Confidence Distribution
    ax6 = axes[1, 2]
    if 'confidence' in df.columns:
        # Create violin plot for confidence
        parts = ax6.violinplot(df['confidence'], vert=False, showmeans=True)
        
        # Customize violin
        for pc in parts['bodies']:
            pc.set_facecolor('lightblue')
            pc.set_alpha(0.7)
        
        parts['cmeans'].set_color('red')
        parts['cmeans'].set_linewidth(2)
        
        ax6.set_xlabel('Confidence Score')
        ax6.set_title('Confidence Distribution')
        ax6.grid(True, alpha=0.3, axis='x')
        ax6.set_yticks([])
        
        # Add quartile lines
        q1, median, q3 = np.percentile(df['confidence'], [25, 50, 75])
        ax6.axvline(x=q1, color='green', linestyle='--', alpha=0.5, label='Q1')
        ax6.axvline(x=median, color='blue', linestyle='--', alpha=0.5, label='Median')
        ax6.axvline(x=q3, color='green', linestyle='--', alpha=0.5, label='Q3')
        ax6.legend(loc='upper right')
    
    plt.suptitle(f'Comparative Analysis of {len(df)} Websites', fontsize=14, fontweight='bold')
    plt.tight_layout()
    
    return fig

# Example batch analysis
print("\n📈 Example Batch Analysis")
print("="*60)

# Test URLs (mix of safe and potentially problematic)
test_urls = [
    "https://httpbin.org",  # Safe test site
    "https://example.com",  # Safe example
    "https://google.com",   # Major site
    "https://github.com",   # Development platform
    # Note: In production, you would test actual websites
    # For demo, we'll simulate with fewer sites
]

# Run batch analysis
batch_df = analyze_multiple_websites(
    urls=test_urls[:2],  # Just test first 2 for demo
    scanner=scanner,
    orchestrator=agent_orchestrator,
    threat_intel=threat_intel
)

# Create comparative visualization
if len(batch_df) > 1:
    print("\n📊 Creating Comparative Analysis Dashboard...")
    comparative_fig = create_comparative_analysis(batch_df)
    plt.show()
else:
    print("⚠️ Need at least 2 websites for comparative analysis")

# %% [markdown]
# ## 9. Threat Intelligence Correlation Analysis

# %%
def correlate_threat_intelligence(urls: List[str],
                                threat_intel: ThreatIntelligenceIngestor) -> pd.DataFrame:
    """
    Correlate threat intelligence across multiple websites.
    """
    
    print(f"\n🔗 THREAT INTELLIGENCE CORRELATION")
    print(f"{'='*60}")
    
    correlation_data = []
    
    for url in urls:
        try:
            parsed_url = urllib.parse.urlparse(url)
            domain = parsed_url.netloc
            
            # Get IP address
            ip_address = None
            try:
                import socket
                ip_address = socket.gethostbyname(domain)
            except:
                pass
            
            # Check various threat feeds
            threats = {
                'url': url,
                'domain': domain,
                'ip_address': ip_address,
                'malware_hits': 0,
                'phishing_hits': 0,
                'exploit_hits': 0,
                'cve_hits': 0,
                'botnet_hits': 0,
                'total_threats': 0
            }
            
            # Check malware IPs
            if ip_address and ip_address in threat_intel.get_malware_ips():
                threats['malware_hits'] += 1
            
            # Check phishing URLs (simplified)
            phishing_urls = threat_intel.get_phishing_urls()
            for phishing_url in phishing_urls[:100]:  # Check first 100
                if domain in phishing_url:
                    threats['phishing_hits'] += 1
            
            # Calculate total threats
            threats['total_threats'] = sum([
                threats['malware_hits'],
                threats['phishing_hits'],
                threats['exploit_hits'],
                threats['cve_hits'],
                threats['botnet_hits']
            ])
            
            correlation_data.append(threats)
            
        except Exception as e:
            print(f"⚠️ Error processing {url}: {e}")
    
    # Create DataFrame
    df = pd.DataFrame(correlation_data)
    
    if not df.empty:
        # Calculate correlation matrix
        threat_columns = ['malware_hits', 'phishing_hits', 'exploit_hits', 
                         'cve_hits', 'botnet_hits', 'total_threats']
        
        if all(col in df.columns for col in threat_columns):
            corr_matrix = df[threat_columns].corr()
            
            # Create correlation heatmap
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
            
            # Heatmap
            im = ax1.imshow(corr_matrix, cmap='coolwarm', vmin=-1, vmax=1, aspect='auto')
            plt.colorbar(im, ax=ax1, label='Correlation Coefficient')
            
            ax1.set_xticks(range(len(threat_columns)))
            ax1.set_yticks(range(len(threat_columns)))
            ax1.set_xticklabels([c.replace('_', '\n') for c in threat_columns], 
                               rotation=45, ha='right')
            ax1.set_yticklabels([c.replace('_', '\n') for c in threat_columns])
            ax1.set_title('Threat Type Correlation Matrix')
            
            # Add correlation values
            for i in range(len(threat_columns)):
                for j in range(len(threat_columns)):
                    ax1.text(j, i, f'{corr_matrix.iloc[i, j]:.2f}', 
                            ha='center', va='center', 
                            color='white' if abs(corr_matrix.iloc[i, j]) > 0.5 else 'black')
            
            # Threat distribution across websites
            threat_counts = df[threat_columns[:-1]].sum()  # Exclude total_threats
            
            colors = plt.cm.Set3(np.linspace(0, 1, len(threat_counts)))
            bars = ax2.bar(range(len(threat_counts)), threat_counts.values, color=colors)
            
            ax2.set_xlabel('Threat Type')
            ax2.set_ylabel('Total Occurrences')
            ax2.set_title('Threat Distribution Across Websites')
            ax2.set_xticks(range(len(threat_counts)))
            ax2.set_xticklabels([c.replace('_', '\n') for c in threat_counts.index], 
                               rotation=45, ha='right')
            ax2.grid(True, alpha=0.3, axis='y')
            
            # Add value labels
            for bar, count in zip(bars, threat_counts.values):
                height = bar.get_height()
                if height > 0:
                    ax2.text(bar.get_x() + bar.get_width()/2., height,
                            f'{int(count)}', ha='center', va='bottom', fontsize=10)
            
            plt.suptitle(f'Threat Intelligence Correlation Analysis ({len(df)} websites)', 
                        fontsize=14, fontweight='bold')
            plt.tight_layout()
            plt.show()
            
            print(f"\n📊 Threat Intelligence Summary:")
            print(f"   Websites analyzed: {len(df)}")
            print(f"   Total threat hits: {df['total_threats'].sum()}")
            print(f"   Websites with threats: {(df['total_threats'] > 0).sum()}")
            print(f"   Average threats per website: {df['total_threats'].mean():.2f}")
            
            # Identify common threats
            if df['total_threats'].sum() > 0:
                print(f"\n🔍 Most Common Threat Types:")
                for threat_type, count in threat_counts.sort_values(ascending=False).items():
                    if count > 0:
                        print(f"   • {threat_type.replace('_', ' ').title()}: {count}")
    
    return df

# Run threat intelligence correlation
print("\n🔗 Running Threat Intelligence Correlation...")
threat_correlation_df = correlate_threat_intelligence(test_urls, threat_intel)

# %% [markdown]
# ## 10. Automated Response Generation

# %%
def generate_automated_responses(df: pd.DataFrame) -> Dict[str, List[Dict]]:
    """
    Generate automated response actions based on analysis results.
    """
    
    print(f"\n🤖 AUTOMATED RESPONSE GENERATION")
    print(f"{'='*60}")
    
    responses = {
        'immediate_actions': [],
        'short_term_actions': [],
        'long_term_actions': [],
        'monitoring_actions': [],
        'reporting_actions': []
    }
    
    for _, row in df.iterrows():
        url = row['url']
        threat_level = row.get('threat_level', 0)
        risk_score = row.get('risk_score', 0)
        recommended_action = row.get('recommended_action', 'MONITOR')
        
        # Generate responses based on threat level and risk
        if threat_level >= 0.8 or risk_score >= 0.8:
            # Critical threats - immediate action required
            responses['immediate_actions'].append({
                'url': url,
                'action': 'BLOCK_TRAFFIC',
                'priority': 'CRITICAL',
                'reason': f'Critical threat detected (threat: {threat_level:.2f}, risk: {risk_score:.2f})',
                'implementation': 'Update firewall rules and WAF policies immediately',
                'verification': 'Confirm traffic blocking and monitor for bypass attempts'
            })
            
            responses['reporting_actions'].append({
                'url': url,
                'action': 'ALERT_SECURITY_TEAM',
                'priority': 'CRITICAL',
                'recipients': ['security-team@company.com', 'soc@company.com'],
                'message': f'CRITICAL: {url} detected with high threat level ({threat_level:.2f})',
                'escalation': 'Notify security management within 15 minutes'
            })
        
        elif threat_level >= 0.6 or risk_score >= 0.6:
            # High threats - short-term actions
            responses['short_term_actions'].append({
                'url': url,
                'action': 'IMPLEMENT_CONTROLS',
                'priority': 'HIGH',
                'controls': [
                    'Enable additional logging',
                    'Implement rate limiting',
                    'Add to monitoring watchlist'
                ],
                'timeline': 'Within 24 hours',
                'owner': 'Security Operations'
            })
        
        elif threat_level >= 0.4:
            # Medium threats - long-term improvements
            responses['long_term_actions'].append({
                'url': url,
                'action': 'SCHEDULE_REVIEW',
                'priority': 'MEDIUM',
                'review_type': 'Security assessment and penetration testing',
                'timeline': 'Within 7 days',
                'owner': 'Security Architecture Team'
            })
        
        else:
            # Low threats - monitoring
            responses['monitoring_actions'].append({
                'url': url,
                'action': 'ENHANCE_MONITORING',
                'priority': 'LOW',
                'monitoring_actions': [
                    'Add to regular scan schedule',
                    'Monitor for behavioral changes',
                    'Track threat intelligence updates'
                ],
                'frequency': 'Weekly review'
            })
    
    # Print summary
    print(f"Generated {len(responses['immediate_actions'])} immediate actions")
    print(f"Generated {len(responses['short_term_actions'])} short-term actions")
    print(f"Generated {len(responses['long_term_actions'])} long-term actions")
    print(f"Generated {len(responses['monitoring_actions'])} monitoring actions")
    print(f"Generated {len(responses['reporting_actions'])} reporting actions")
    
    # Create response visualization
    fig, ax = plt.subplots(figsize=(10, 6))
    
    action_categories = list(responses.keys())
    action_counts = [len(actions) for actions in responses.values()]
    
    colors = ['red', 'orange', 'yellow', 'lightblue', 'lightgreen']
    bars = ax.bar(action_categories, action_counts, color=colors, alpha=0.7)
    
    ax.set_xlabel('Action Category')
    ax.set_ylabel('Number of Actions')
    ax.set_title('Automated Response Actions by Category')
    ax.set_xticklabels([c.replace('_', '\n') for c in action_categories], rotation=45, ha='right')
    ax.grid(True, alpha=0.3, axis='y')
    
    # Add value labels
    for bar, count in zip(bars, action_counts):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
               f'{count}', ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    return responses

# Generate automated responses
if not batch_df.empty:
    print("\n🤖 Generating Automated Responses...")
    automated_responses = generate_automated_responses(batch_df)
    
    # Display example actions
    if automated_responses['immediate_actions']:
        print(f"\n🚨 EXAMPLE IMMEDIATE ACTION:")
        action = automated_responses['immediate_actions'][0]
        print(f"   URL: {action['url']}")
        print(f"   Action: {action['action']}")
        print(f"   Priority: {action['priority']}")
        print(f"   Reason: {action['reason']}")
    
    if automated_responses['reporting_actions']:
        print(f"\n📧 EXAMPLE REPORTING ACTION:")
        action = automated_responses['reporting_actions'][0]
        print(f"   Recipients: {', '.join(action['recipients'])}")
        print(f"   Message: {action['message']}")

# %% [markdown]
# ## 11. Exporting Analysis Results

# %%
def export_analysis_results(report: Dict, 
                          batch_df: pd.DataFrame,
                          threat_correlation_df: pd.DataFrame,
                          automated_responses: Dict,
                          export_dir: str = "exports/threat_analysis/"):
    """
    Export all analysis results to various formats.
    """
    
    import os
    import json
    import csv
    from datetime import datetime
    
    # Create export directory
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    export_path = os.path.join(export_dir, timestamp)
    os.makedirs(export_path, exist_ok=True)
    
    print(f"\n📤 Exporting Analysis Results to: {export_path}")
    
    # 1. Export individual report as JSON
    report_path = os.path.join(export_path, "security_report.json")
    with open(report_path, 'w') as f:
        json.dump(report, f, indent=2, default=str)
    print(f"   ✅ Security report: {report_path}")
    
    # 2. Export batch analysis as CSV
    if not batch_df.empty:
        csv_path = os.path.join(export_path, "batch_analysis.csv")
        batch_df.to_csv(csv_path, index=False)
        print(f"   ✅ Batch analysis: {csv_path}")
    
    # 3. Export threat correlation as CSV
    if not threat_correlation_df.empty:
        threat_path = os.path.join(export_path, "threat_correlation.csv")
        threat_correlation_df.to_csv(threat_path, index=False)
        print(f"   ✅ Threat correlation: {threat_path}")
    
    # 4. Export automated responses as JSON
    responses_path = os.path.join(export_path, "automated_responses.json")
    with open(responses_path, 'w') as f:
        json.dump(automated_responses, f, indent=2, default=str)
    print(f"   ✅ Automated responses: {responses_path}")
    
    # 5. Export summary report
    summary = {
        'export_timestamp': datetime.now().isoformat(),
        'total_websites_analyzed': len(batch_df) if not batch_df.empty else 1,
        'average_threat_level': batch_df['threat_level'].mean() if not batch_df.empty else report['executive_summary']['threat_level'],
        'average_risk_score': batch_df['risk_score'].mean() if not batch_df.empty else report['executive_summary']['risk_score'],
        'total_vulnerabilities': batch_df['vulnerability_count'].sum() if not batch_df.empty else len(report['vulnerabilities']),
        'critical_vulnerabilities': batch_df['critical_vulns'].sum() if not batch_df.empty else len([v for v in report['vulnerabilities'] if v.get('severity') == 'CRITICAL']),
        'high_vulnerabilities': batch_df['high_vulns'].sum() if not batch_df.empty else len([v for v in report['vulnerabilities'] if v.get('severity') == 'HIGH']),
        'websites_requiring_immediate_action': len(automated_responses.get('immediate_actions', [])),
        'analysis_duration_seconds': batch_df['analysis_duration'].mean() if not batch_df.empty else report['metadata']['analysis_duration']
    }
    
    summary_path = os.path.join(export_path, "analysis_summary.json")
    with open(summary_path, 'w') as f:
        json.dump(summary, f, indent=2)
    print(f"   ✅ Analysis summary: {summary_path}")
    
    # 6. Create HTML report
    html_path = os.path.join(export_path, "security_report.html")
    create_html_report(report, batch_df, summary, html_path)
    print(f"   ✅ HTML report: {html_path}")
    
    # 7. Save visualizations
    vis_path = os.path.join(export_path, "dashboard.png")
    dashboard_fig.savefig(vis_path, dpi=300, bbox_inches='tight')
    print(f"   ✅ Dashboard visualization: {vis_path}")
    
    print(f"\n📁 All results exported successfully to: {export_path}")
    
    return export_path

def create_html_report(report: Dict, 
                      batch_df: pd.DataFrame,
                      summary: Dict,
                      output_path: str):
    """Create HTML report for easy viewing."""
    
    html_template = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>CyberGuard Security Report</title>
        <style>
            body { font-family: Arial, sans-serif; margin: 40px; }
            .header { background: #2c3e50; color: white; padding: 20px; border-radius: 5px; }
            .summary { background: #ecf0f1; padding: 20px; border-radius: 5px; margin: 20px 0; }
            .section { margin: 30px 0; }
            .card { background: white; padding: 20px; border-radius: 5px; box-shadow: 0 2px 5px rgba(0,0,0,0.1); margin: 10px 0; }
            .critical { color: #e74c3c; font-weight: bold; }
            .high { color: #e67e22; }
            .medium { color: #f39c12; }
            .low { color: #3498db; }
            table { width: 100%; border-collapse: collapse; }
            th, td { padding: 10px; text-align: left; border-bottom: 1px solid #ddd; }
            th { background: #f2f2f2; }
            .badge { padding: 5px 10px; border-radius: 3px; color: white; font-size: 12px; }
            .badge-critical { background: #e74c3c; }
            .badge-high { background: #e67e22; }
            .badge-medium { background: #f39c12; }
            .badge-low { background: #3498db; }
        </style>
    </head>
    <body>
        <div class="header">
            <h1>🚀 CyberGuard Security Analysis Report</h1>
            <p>Generated: {timestamp}</p>
        </div>
        
        <div class="summary">
            <h2>📊 Executive Summary</h2>
            <p><strong>URL Analyzed:</strong> {url}</p>
            <p><strong>Threat Level:</strong> <span class="{threat_class}">{threat_level} ({threat_category})</span></p>
            <p><strong>Risk Score:</strong> {risk_score}/1.0</p>
            <p><strong>Confidence:</strong> {confidence}/1.0</p>
            <p><strong>Recommended Action:</strong> {recommended_action}</p>
        </div>
        
        <div class="section">
            <h2>🔍 Key Findings</h2>
            <div class="card">
                <ul>
                    {key_findings}
                </ul>
            </div>
        </div>
        
        <div class="section">
            <h2>📈 Vulnerability Analysis</h2>
            <div class="card">
                <p><strong>Total Vulnerabilities:</strong> {total_vulns}</p>
                <p><strong>Critical:</strong> {critical_vulns}</p>
                <p><strong>High:</strong> {high_vulns}</p>
                <p><strong>Medium:</strong> {medium_vulns}</p>
                <p><strong>Low:</strong> {low_vulns}</p>
            </div>
        </div>
        
        <div class="section">
            <h2>💡 Top Recommendations</h2>
            <div class="card">
                <table>
                    <tr>
                        <th>Priority</th>
                        <th>Recommendation</th>
                        <th>Category</th>
                    </tr>
                    {recommendations}
                </table>
            </div>
        </div>
        
        <div class="section">
            <h2>🤖 Agent Analysis</h2>
            <div class="card">
                <p><strong>Number of Agents:</strong> {agent_count}</p>
                <p><strong>Agent Agreement:</strong> {agent_agreement}%</p>
                <p><strong>Requires Human Review:</strong> {requires_review}</p>
            </div>
        </div>
        
        <div class="section">
            <h2>📋 Batch Analysis Summary</h2>
            <div class="card">
                <p><strong>Total Websites Analyzed:</strong> {batch_count}</p>
                <p><strong>Average Threat Level:</strong> {avg_threat}</p>
                <p><strong>Average Risk Score:</strong> {avg_risk}</p>
                <p><strong>Total Critical Vulnerabilities:</strong> {total_critical}</p>
            </div>
        </div>
        
        <div class="section">
            <h2>📄 Report Metadata</h2>
            <div class="card">
                <p><strong>Analysis Start:</strong> {analysis_start}</p>
                <p><strong>Analysis End:</strong> {analysis_end}</p>
                <p><strong>Analysis Duration:</strong> {analysis_duration} seconds</p>
                <p><strong>Report Version:</strong> {report_version}</p>
                <p><strong>CyberGuard Version:</strong> {cyberguard_version}</p>
            </div>
        </div>
        
        <div class="section">
            <p><em>Report generated by CyberGuard - Intelligent Web Threat Analysis & Defense Platform</em></p>
            <p><em>For questions or concerns, contact: security@cyberguard.ai</em></p>
        </div>
    </body>
    </html>
    """
    
    # Prepare data for HTML template
    threat_level = report['executive_summary']['threat_level']
    threat_category = report['executive_summary']['threat_category']
    
    # Determine threat class for CSS
    if threat_level >= 0.8:
        threat_class = 'critical'
    elif threat_level >= 0.6:
        threat_class = 'high'
    elif threat_level >= 0.4:
        threat_class = 'medium'
    else:
        threat_class = 'low'
    
    # Count vulnerabilities by severity
    vulns = report['vulnerabilities']
    critical_vulns = len([v for v in vulns if v.get('severity') == 'CRITICAL'])
    high_vulns = len([v for v in vulns if v.get('severity') == 'HIGH'])
    medium_vulns = len([v for v in vulns if v.get('severity') == 'MEDIUM'])
    low_vulns = len([v for v in vulns if v.get('severity') == 'LOW'])
    total_vulns = len(vulns)
    
    # Prepare key findings HTML
    key_findings_html = ""
    for finding in report['executive_summary']['key_findings']:
        key_findings_html += f"<li>{finding}</li>\n"
    
    # Prepare recommendations HTML
    recommendations_html = ""
    for rec in report['recommendations'][:5]:  # Top 5 recommendations
        priority_class = f"badge-{rec['priority'].lower()}" if rec['priority'].lower() in ['critical', 'high', 'medium', 'low'] else ""
        recommendations_html += f"""
        <tr>
            <td><span class="badge {priority_class}">{rec['priority']}</span></td>
            <td>{rec['recommendation']}</td>
            <td>{rec.get('category', 'General')}</td>
        </tr>
        """
    
    # Fill template
    html_content = html_template.format(
        timestamp=report['metadata']['analysis_end'],
        url=report['metadata']['url'],
        threat_class=threat_class,
        threat_level=f"{threat_level:.2f}",
        threat_category=threat_category,
        risk_score=f"{report['executive_summary']['risk_score']:.2f}",
        confidence=f"{report['executive_summary']['confidence']:.2f}",
        recommended_action=report['executive_summary']['recommended_action'],
        key_findings=key_findings_html,
        total_vulns=total_vulns,
        critical_vulns=critical_vulns,
        high_vulns=high_vulns,
        medium_vulns=medium_vulns,
        low_vulns=low_vulns,
        recommendations=recommendations_html,
        agent_count=len(report['agent_contributions']),
        agent_agreement=f"{report['executive_summary'].get('agent_agreement', 80)}",  # Example
        requires_review="Yes" if report['requires_human_review'] else "No",
        batch_count=summary.get('total_websites_analyzed', 1),
        avg_threat=f"{summary.get('average_threat_level', 0):.2f}",
        avg_risk=f"{summary.get('average_risk_score', 0):.2f}",
        total_critical=summary.get('critical_vulnerabilities', 0),
        analysis_start=report['metadata']['analysis_start'],
        analysis_end=report['metadata']['analysis_end'],
        analysis_duration=f"{report['metadata']['analysis_duration']:.1f}",
        report_version=report['metadata']['report_version'],
        cyberguard_version=report['metadata']['cyberguard_version']
    )
    
    # Write HTML file
    with open(output_path, 'w') as f:
        f.write(html_content)

# Export results
print("\n📤 Exporting Analysis Results...")
try:
    export_path = export_analysis_results(
        report=report,
        batch_df=batch_df,
        threat_correlation_df=threat_correlation_df,
        automated_responses=automated_responses if 'automated_responses' in locals() else {}
    )
    print(f"\n✅ Export complete! Files saved to: {export_path}")
except Exception as e:
    print(f"⚠️ Export error: {e}")

# %% [markdown]
# ## 12. Real-time Threat Monitoring Simulation

# %%
def simulate_real_time_monitoring(scanner: WebSecurityScanner,
                                orchestrator: AgentOrchestrator,
                                threat_intel: ThreatIntelligenceIngestor,
                                duration_minutes: int = 5):
    """
    Simulate real-time threat monitoring and alerting.
    """
    
    import time
    import random
    
    print(f"\n🕒 SIMULATING REAL-TIME THREAT MONITORING")
    print(f"   Duration: {duration_minutes} minutes")
    print(f"{'='*60}")
    
    # Simulated websites to monitor
    monitored_sites = [
        "https://httpbin.org",
        "https://example.com",
        "https://testphp.vulnweb.com",  # Vulnerable test site
        "https://google.com"
    ]
    
    # Alert thresholds
    ALERT_THRESHOLD = 0.7  # Threat level threshold for alerts
    CRITICAL_THRESHOLD = 0.9  # Critical threat threshold
    
    # Monitoring variables
    start_time = time.time()
    end_time = start_time + (duration_minutes * 60)
    scan_interval = 30  # Scan every 30 seconds
    alerts = []
    
    print(f"\n📡 Monitoring {len(monitored_sites)} websites...")
    print(f"   Alert threshold: {ALERT_THRESHOLD}")
    print(f"   Critical threshold: {CRITICAL_THRESHOLD}")
    print(f"   Scan interval: {scan_interval} seconds")
    print()
    
    scan_count = 0
    
    while time.time() < end_time:
        scan_count += 1
        current_time = datetime.now().strftime("%H:%M:%S")
        
        print(f"\n[{current_time}] Scan #{scan_count}")
        print("-" * 40)
        
        # Simulate random website selection (in reality, would scan all)
        site_to_scan = random.choice(monitored_sites)
        
        try:
            # Perform security analysis
            report = analyze_website_security(
                url=site_to_scan,
                scanner=scanner,
                orchestrator=orchestrator,
                threat_intel=threat_intel
            )
            
            threat_level = report['executive_summary']['threat_level']
            risk_score = report['executive_summary']['risk_score']
            
            print(f"   Site: {site_to_scan}")
            print(f"   Threat Level: {threat_level:.2f}")
            print(f"   Risk Score: {risk_score:.2f}")
            
            # Check for alerts
            if threat_level >= CRITICAL_THRESHOLD:
                alert_level = "🚨 CRITICAL"
                alert_color = "\033[91m"  # Red
            elif threat_level >= ALERT_THRESHOLD:
                alert_level = "⚠️  HIGH"
                alert_color = "\033[93m"  # Yellow
            else:
                alert_level = "✅ Normal"
                alert_color = "\033[92m"  # Green
            
            print(f"   Status: {alert_color}{alert_level}\033[0m")
            
            # Generate alert if threshold exceeded
            if threat_level >= ALERT_THRESHOLD:
                alert = {
                    'timestamp': current_time,
                    'site': site_to_scan,
                    'threat_level': threat_level,
                    'risk_score': risk_score,
                    'alert_level': 'CRITICAL' if threat_level >= CRITICAL_THRESHOLD else 'HIGH',
                    'scan_id': scan_count
                }
                alerts.append(alert)
                
                print(f"   {alert_color}ALERT GENERATED!\033[0m")
                print(f"   Threat details saved for investigation")
            
            # Simulate some random threat events
            if random.random() < 0.2:  # 20% chance of simulated event
                event_types = ['SQL injection attempt', 'XSS payload detected', 
                             'Credential stuffing', 'Port scanning detected']
                event = random.choice(event_types)
                print(f"   📡 Simulated event: {event}")
        
        except Exception as e:
            print(f"   ⚠️ Scan error: {e}")
        
        # Wait for next scan
        if time.time() + scan_interval < end_time:
            time.sleep(scan_interval)
    
    # Monitoring summary
    print(f"\n{'='*60}")
    print("MONITORING SESSION COMPLETE")
    print(f"{'='*60}")
    print(f"Total scans: {scan_count}")
    print(f"Total alerts: {len(alerts)}")
    
    if alerts:
        print(f"\n📋 ALERT SUMMARY:")
        for alert in alerts:
            print(f"  [{alert['timestamp']}] {alert['site']}")
            print(f"     Threat: {alert['threat_level']:.2f} ({alert['alert_level']})")
        
        # Create alert timeline
        fig, ax = plt.subplots(figsize=(10, 6))
        
        # Convert timestamps to datetime objects for plotting
        alert_times = [datetime.strptime(f"2024-01-01 {a['timestamp']}", "%Y-%m-%d %H:%M:%S") 
                      for a in alerts]
        alert_levels = [1 if a['alert_level'] == 'CRITICAL' else 0.5 for a in alerts]
        alert_sites = [a['site'] for a in alerts]
        
        # Create color map for sites
        unique_sites = list(set(alert_sites))
        site_colors = plt.cm.Set1(np.linspace(0, 1, len(unique_sites)))
        site_to_color = {site: color for site, color in zip(unique_sites, site_colors)}
        
        # Plot alerts
        for time, level, site in zip(alert_times, alert_levels, alert_sites):
            color = site_to_color[site]
            ax.scatter(time, level, color=color, s=100, alpha=0.7, label=site)
        
        # Customize plot
        ax.set_xlabel('Time')
        ax.set_ylabel('Alert Level')
        ax.set_title('Threat Alert Timeline')
        ax.set_yticks([0.5, 1])
        ax.set_yticklabels(['HIGH', 'CRITICAL'])
        ax.grid(True, alpha=0.3)
        
        # Add legend (without duplicates)
        handles, labels = ax.get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        ax.legend(by_label.values(), by_label.keys(), loc='upper right')
        
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
    else:
        print("\n✅ No alerts generated during monitoring session")
    
    return alerts

# Run real-time monitoring simulation
print("\n🕒 Starting Real-time Monitoring Simulation...")
try:
    alerts = simulate_real_time_monitoring(
        scanner=scanner,
        orchestrator=agent_orchestrator,
        threat_intel=threat_intel,
        duration_minutes=2  # Short simulation for demo
    )
except KeyboardInterrupt:
    print("\n⏹️ Monitoring simulation interrupted by user")
except Exception as e:
    print(f"\n⚠️ Monitoring simulation error: {e}")

# %% [markdown]
# ## 13. Threat Hunting and Proactive Detection

# %%
def proactive_threat_hunting(scanner: WebSecurityScanner,
                           orchestrator: AgentOrchestrator,
                           threat_intel: ThreatIntelligenceIngestor):
    """
    Proactive threat hunting for emerging threats and zero-day vulnerabilities.
    """
    
    print(f"\n🔍 PROACTIVE THREAT HUNTING")
    print(f"{'='*60}")
    
    # Define hunting scenarios
    hunting_scenarios = [
        {
            'name': 'Zero-Day Exploit Detection',
            'description': 'Hunt for signs of unknown exploits and zero-day vulnerabilities',
            'techniques': [
                'Anomalous payload patterns',
                'Unusual request sequences',
                'Behavioral deviation analysis',
                'Machine learning anomaly detection'
            ]
        },
        {
            'name': 'Advanced Persistent Threat (APT)',
            'description': 'Hunt for sophisticated, targeted attacks',
            'techniques': [
                'Command and control communication patterns',
                'Data exfiltration detection',
                'Lateral movement indicators',
                'Persistence mechanism analysis'
            ]
        },
        {
            'name': 'Supply Chain Attacks',
            'description': 'Hunt for compromised dependencies and third-party components',
            'techniques': [
                'Dependency version analysis',
                'Code signature verification',
                'Behavioral analysis of third-party scripts',
                'Reputation scoring of external resources'
            ]
        },
        {
            'name': 'Credential-Based Attacks',
            'description': 'Hunt for credential theft and abuse',
            'techniques': [
                'Credential stuffing pattern detection',
                'Brute force attack identification',
                'Credential reuse analysis',
                'Multi-factor authentication bypass attempts'
            ]
        }
    ]
    
    # Simulate threat hunting results
    hunting_results = []
    
    for scenario in hunting_scenarios:
        print(f"\n🎯 Hunting Scenario: {scenario['name']}")
        print(f"   Description: {scenario['description']}")
        
        # Simulate hunting with random results
        techniques_used = random.sample(scenario['techniques'], 
                                       min(2, len(scenario['techniques'])))
        
        # Generate simulated findings
        findings = []
        if random.random() < 0.3:  # 30% chance of finding something
            finding_types = [
                'Suspicious pattern detected',
                'Anomalous behavior identified',
                'Potential indicator of compromise',
                'Security control bypass attempt'
            ]
            findings = random.sample(finding_types, random.randint(1, 2))
        
        # Calculate threat score for this scenario
        threat_score = random.random() * 0.5  # Lower scores for proactive hunting
        if findings:
            threat_score += 0.3
        
        # Determine action
        if threat_score > 0.6:
            action = 'IMMEDIATE_INVESTIGATION'
            priority = 'HIGH'
        elif threat_score > 0.4:
            action = 'ENHANCE_MONITORING'
            priority = 'MEDIUM'
        else:
            action = 'CONTINUE_HUNTING'
            priority = 'LOW'
        
        scenario_result = {
            'scenario': scenario['name'],
            'threat_score': threat_score,
            'techniques_used': techniques_used,
            'findings': findings,
            'action': action,
            'priority': priority,
            'confidence': random.random() * 0.3 + 0.5  # 50-80% confidence
        }
        
        hunting_results.append(scenario_result)
        
        # Print scenario results
        print(f"   Threat Score: {threat_score:.2f}")
        print(f"   Techniques Used: {', '.join(techniques_used)}")
        if findings:
            print(f"   Findings: {', '.join(findings)}")
        else:
            print(f"   Findings: No significant findings")
        print(f"   Recommended Action: {action} ({priority} priority)")
        print(f"   Confidence: {scenario_result['confidence']:.2f}")
    
    # Create hunting results visualization
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # 1. Threat scores by scenario
    ax1 = axes[0]
    scenarios = [r['scenario'] for r in hunting_results]
    scores = [r['threat_score'] for r in hunting_results]
    colors = ['red' if s > 0.6 else 'orange' if s > 0.4 else 'green' for s in scores]
    
    bars = ax1.barh(scenarios, scores, color=colors, alpha=0.7)
    ax1.set_xlabel('Threat Score')
    ax1.set_title('Threat Hunting Results by Scenario')
    ax1.set_xlim(0, 1)
    ax1.grid(True, alpha=0.3, axis='x')
    
    # Add value labels
    for bar, score in zip(bars, scores):
        width = bar.get_width()
        ax1.text(width + 0.02, bar.get_y() + bar.get_height()/2,
                f'{score:.2f}', ha='left', va='center', fontsize=9)
    
    # Add threshold lines
    ax1.axvline(x=0.4, color='yellow', linestyle='--', alpha=0.5, label='Monitor Threshold')
    ax1.axvline(x=0.6, color='red', linestyle='--', alpha=0.5, label='Investigate Threshold')
    ax1.legend()
    
    # 2. Action priority distribution
    ax2 = axes[1]
    priorities = ['HIGH', 'MEDIUM', 'LOW']
    priority_counts = {p: 0 for p in priorities}
    
    for result in hunting_results:
        priority_counts[result['priority']] += 1
    
    # Create donut chart
    wedges, texts, autotexts = ax2.pie(
        [priority_counts[p] for p in priorities],
        labels=priorities,
        colors=['red', 'orange', 'green'],
        autopct='%1.1f%%',
        startangle=90,
        pctdistance=0.85
    )
    
    # Draw circle for donut effect
    centre_circle = plt.Circle((0, 0), 0.70, fc='white')
    ax2.add_artist(centre_circle)
    
    ax2.set_title('Action Priority Distribution')
    
    # Enhance autotexts
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')
    
    plt.suptitle('Proactive Threat Hunting Results', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Generate hunting recommendations
    print(f"\n{'='*60}")
    print("THREAT HUNTING RECOMMENDATIONS")
    print(f"{'='*60}")
    
    high_priority_results = [r for r in hunting_results if r['priority'] == 'HIGH']
    if high_priority_results:
        print(f"\n🚨 IMMEDIATE ACTIONS REQUIRED:")
        for result in high_priority_results:
            print(f"  • {result['scenario']}: {result['action']}")
            if result['findings']:
                print(f"    Findings: {', '.join(result['findings'])}")
    
    medium_priority_results = [r for r in hunting_results if r['priority'] == 'MEDIUM']
    if medium_priority_results:
        print(f"\n⚠️  ENHANCED MONITORING RECOMMENDED:")
        for result in medium_priority_results:
            print(f"  • {result['scenario']}: Increase monitoring frequency")
    
    print(f"\n📊 Hunting Effectiveness Metrics:")
    total_scenarios = len(hunting_results)
    scenarios_with_findings = len([r for r in hunting_results if r['findings']])
    avg_threat_score = np.mean([r['threat_score'] for r in hunting_results])
    avg_confidence = np.mean([r['confidence'] for r in hunting_results])
    
    print(f"  • Scenarios with findings: {scenarios_with_findings}/{total_scenarios}")
    print(f"  • Average threat score: {avg_threat_score:.2f}")
    print(f"  • Average confidence: {avg_confidence:.2f}")
    print(f"  • Hunting coverage: {len(hunting_scenarios)} distinct scenarios")
    
    return hunting_results

# Run proactive threat hunting
print("\n🔍 Starting Proactive Threat Hunting...")
hunting_results = proactive_threat_hunting(scanner, agent_orchestrator, threat_intel)

# %% [markdown]
# ## 14. Machine Learning for Threat Prediction

# %%
def threat_prediction_ml(scanner: WebSecurityScanner,
                       orchestrator: AgentOrchestrator,
                       historical_data: pd.DataFrame = None):
    """
    Machine learning-based threat prediction and risk forecasting.
    """
    
    print(f"\n🤖 MACHINE LEARNING THREAT PREDICTION")
    print(f"{'='*60}")
    
    # If no historical data provided, create simulated data
    if historical_data is None:
        print("Generating simulated historical data...")
        
        # Create simulated historical data
        np.random.seed(42)
        n_samples = 1000
        
        historical_data = pd.DataFrame({
            'url_length': np.random.randint(10, 100, n_samples),
            'num_parameters': np.random.randint(0, 20, n_samples),
            'https_enabled': np.random.choice([0, 1], n_samples, p=[0.2, 0.8]),
            'security_headers_score': np.random.uniform(0, 1, n_samples),
            'response_time_ms': np.random.exponential(500, n_samples),
            'status_code': np.random.choice([200, 404, 500, 301, 302], n_samples),
            'content_length': np.random.randint(1000, 1000000, n_samples),
            'threat_intel_matches': np.random.poisson(0.5, n_samples),
            # Target variables
            'threat_level': np.random.beta(2, 5, n_samples),  # Most sites low threat
            'was_attacked': np.random.choice([0, 1], n_samples, p=[0.9, 0.1])  # 10% attacked
        })
        
        # Add some correlations
        historical_data['threat_level'] += historical_data['threat_intel_matches'] * 0.1
        historical_data['threat_level'] += (1 - historical_data['https_enabled']) * 0.2
        historical_data['threat_level'] = np.clip(historical_data['threat_level'], 0, 1)
        
        historical_data['was_attacked'] = (
            (historical_data['threat_level'] > 0.7) | 
            (historical_data['threat_intel_matches'] > 2)
        ).astype(int)
    
    print(f"Historical data: {len(historical_data)} samples")
    print(f"Features: {len(historical_data.columns)}")
    
    # Prepare data for ML
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, r2_score, mean_squared_error
    
    # Features and targets
    feature_columns = ['url_length', 'num_parameters', 'https_enabled', 
                      'security_headers_score', 'response_time_ms', 
                      'status_code', 'content_length', 'threat_intel_matches']
    
    X = historical_data[feature_columns]
    y_threat = historical_data['threat_level']  # Regression target
    y_attack = historical_data['was_attacked']  # Classification target
    
    # Split data
    X_train, X_test, y_threat_train, y_threat_test, y_attack_train, y_attack_test = train_test_split(
        X, y_threat, y_attack, test_size=0.2, random_state=42
    )
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Train models
    print("\nTraining machine learning models...")
    
    # 1. Threat level prediction (regression)
    threat_model = RandomForestRegressor(n_estimators=100, random_state=42)
    threat_model.fit(X_train_scaled, y_threat_train)
    threat_pred = threat_model.predict(X_test_scaled)
    
    # 2. Attack prediction (classification)
    attack_model = RandomForestClassifier(n_estimators=100, random_state=42)
    attack_model.fit(X_train_scaled, y_attack_train)
    attack_pred = attack_model.predict(X_test_scaled)
    attack_proba = attack_model.predict_proba(X_test_scaled)[:, 1]
    
    # Evaluate models
    print(f"\n📊 MODEL EVALUATION")
    print(f"{'-'*40}")
    
    # Threat level model evaluation
    threat_r2 = r2_score(y_threat_test, threat_pred)
    threat_mse = mean_squared_error(y_threat_test, threat_pred)
    
    print(f"Threat Level Prediction (Regression):")
    print(f"  R² Score: {threat_r2:.3f}")
    print(f"  Mean Squared Error: {threat_mse:.3f}")
    
    # Attack prediction model evaluation
    attack_accuracy = accuracy_score(y_attack_test, attack_pred)
    attack_precision = precision_score(y_attack_test, attack_pred)
    attack_recall = recall_score(y_attack_test, attack_pred)
    attack_f1 = f1_score(y_attack_test, attack_pred)
    
    print(f"\nAttack Prediction (Classification):")
    print(f"  Accuracy: {attack_accuracy:.3f}")
    print(f"  Precision: {attack_precision:.3f}")
    print(f"  Recall: {attack_recall:.3f}")
    print(f"  F1-Score: {attack_f1:.3f}")
    
    # Feature importance analysis
    feature_importance = pd.DataFrame({
        'feature': feature_columns,
        'importance_threat': threat_model.feature_importances_,
        'importance_attack': attack_model.feature_importances_
    }).sort_values('importance_attack', ascending=False)
    
    print(f"\n🔍 FEATURE IMPORTANCE:")
    for _, row in feature_importance.iterrows():
        print(f"  {row['feature']}:")
        print(f"    Threat prediction: {row['importance_threat']:.3f}")
        print(f"    Attack prediction: {row['importance_attack']:.3f}")
    
    # Create ML visualization
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Threat level prediction vs actual
    ax1 = axes[0, 0]
    ax1.scatter(y_threat_test, threat_pred, alpha=0.6, s=50)
    ax1.plot([0, 1], [0, 1], 'r--', alpha=0.5, label='Perfect Prediction')
    ax1.set_xlabel('Actual Threat Level')
    ax1.set_ylabel('Predicted Threat Level')
    ax1.set_title('Threat Level Prediction Accuracy')
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    # Add R² text
    ax1.text(0.05, 0.9, f'R² = {threat_r2:.3f}', transform=ax1.transAxes,
            fontsize=12, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # 2. ROC curve for attack prediction
    from sklearn.metrics import roc_curve, auc
    
    ax2 = axes[0, 1]
    fpr, tpr, _ = roc_curve(y_attack_test, attack_proba)
    roc_auc = auc(fpr, tpr)
    
    ax2.plot(fpr, tpr, color='darkorange', lw=2, 
            label=f'ROC curve (AUC = {roc_auc:.3f})')
    ax2.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', alpha=0.5)
    ax2.set_xlabel('False Positive Rate')
    ax2.set_ylabel('True Positive Rate')
    ax2.set_title('ROC Curve - Attack Prediction')
    ax2.grid(True, alpha=0.3)
    ax2.legend(loc='lower right')
    
    # 3. Feature importance
    ax3 = axes[1, 0]
    x_pos = np.arange(len(feature_importance))
    width = 0.35
    
    bars1 = ax3.bar(x_pos - width/2, feature_importance['importance_threat'], 
                   width, label='Threat Prediction', alpha=0.7)
    bars2 = ax3.bar(x_pos + width/2, feature_importance['importance_attack'], 
                   width, label='Attack Prediction', alpha=0.7)
    
    ax3.set_xlabel('Feature')
    ax3.set_ylabel('Importance')
    ax3.set_title('Feature Importance Comparison')
    ax3.set_xticks(x_pos)
    ax3.set_xticklabels([f.replace('_', '\n') for f in feature_importance['feature']], 
                       rotation=45, ha='right', fontsize=9)
    ax3.legend()
    ax3.grid(True, alpha=0.3, axis='y')
    
    # 4. Prediction error distribution
    ax4 = axes[1, 1]
    threat_errors = y_threat_test - threat_pred
    
    ax4.hist(threat_errors, bins=30, alpha=0.7, color='blue', edgecolor='black')
    ax4.set_xlabel('Prediction Error (Actual - Predicted)')
    ax4.set_ylabel('Frequency')
    ax4.set_title('Threat Level Prediction Error Distribution')
    ax4.grid(True, alpha=0.3)
    
    # Add error statistics
    error_mean = threat_errors.mean()
    error_std = threat_errors.std()
    
    ax4.axvline(x=error_mean, color='red', linestyle='--', 
               label=f'Mean: {error_mean:.3f}')
    ax4.axvline(x=error_mean - error_std, color='orange', linestyle=':', 
               alpha=0.5, label=f'±1 STD')
    ax4.axvline(x=error_mean + error_std, color='orange', linestyle=':', alpha=0.5)
    ax4.legend()
    
    plt.suptitle('Machine Learning Threat Prediction Analysis', 
                fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Make predictions on new data
    print(f"\n🎯 MAKING PREDICTIONS ON NEW DATA")
    print(f"{'-'*40}")
    
    # Create new test cases
    new_cases = pd.DataFrame([{
        'url_length': 45,
        'num_parameters': 3,
        'https_enabled': 1,
        'security_headers_score': 0.8,
        'response_time_ms': 350,
        'status_code': 200,
        'content_length': 15000,
        'threat_intel_matches': 0
    }, {
        'url_length': 120,
        'num_parameters': 15,
        'https_enabled': 0,
        'security_headers_score': 0.2,
        'response_time_ms': 1200,
        'status_code': 404,
        'content_length': 500,
        'threat_intel_matches': 3
    }])
    
    # Scale and predict
    new_cases_scaled = scaler.transform(new_cases)
    
    new_threat_pred = threat_model.predict(new_cases_scaled)
    new_attack_pred = attack_model.predict(new_cases_scaled)
    new_attack_proba = attack_model.predict_proba(new_cases_scaled)[:, 1]
    
    for i, (threat, attack, proba) in enumerate(zip(new_threat_pred, new_attack_pred, new_attack_proba)):
        print(f"\nCase {i+1}:")
        print(f"  Predicted Threat Level: {threat:.3f}")
        print(f"  Attack Probability: {proba:.3f}")
        print(f"  Attack Prediction: {'YES' if attack == 1 else 'NO'}")
        
        # Risk assessment
        if threat > 0.7 or proba > 0.7:
            risk_level = "🚨 HIGH RISK"
            recommendation = "Immediate investigation required"
        elif threat > 0.4 or proba > 0.4:
            risk_level = "⚠️  MEDIUM RISK"
            recommendation = "Enhanced monitoring recommended"
        else:
            risk_level = "✅ LOW RISK"
            recommendation = "Normal monitoring sufficient"
        
        print(f"  Risk Assessment: {risk_level}")
        print(f"  Recommendation: {recommendation}")
    
    return {
        'threat_model': threat_model,
        'attack_model': attack_model,
        'scaler': scaler,
        'feature_importance': feature_importance,
        'performance': {
            'threat_r2': threat_r2,
            'threat_mse': threat_mse,
            'attack_accuracy': attack_accuracy,
            'attack_precision': attack_precision,
            'attack_recall': attack_recall,
            'attack_f1': attack_f1,
            'roc_auc': roc_auc
        }
    }

# Run ML threat prediction
print("\n🤖 Running Machine Learning Threat Prediction...")
ml_results = threat_prediction_ml(scanner, agent_orchestrator)

# %% [markdown]
# ## 15. Integration with Security Operations

# %%
def security_operations_integration(report: Dict,
                                  ml_results: Dict,
                                  hunting_results: List):
    """
    Integrate analysis results with security operations workflows.
    """
    
    print(f"\n🛡️ SECURITY OPERATIONS INTEGRATION")
    print(f"{'='*60}")
    
    # Create integrated security dashboard
    integrated_data = {
        'threat_analysis': report,
        'ml_predictions': ml_results,
        'threat_hunting': hunting_results,
        'integration_timestamp': datetime.now().isoformat(),
        'operations_status': {
            'monitoring_active': True,
            'alerting_enabled': True,
            'automated_response': True,
            'human_review_required': report.get('requires_human_review', False)
        }
    }
    
    # Generate operations actions
    operations_actions = []
    
    # 1. Based on threat analysis
    if report['executive_summary']['threat_category'] in ['CRITICAL', 'HIGH']:
        operations_actions.append({
            'type': 'incident_response',
            'priority': 'HIGH',
            'action': 'CREATE_INCIDENT',
            'details': {
                'incident_title': f"Security Threat: {report['metadata']['url']}",
                'severity': report['executive_summary']['threat_category'],
                'description': f"Threat level {report['executive_summary']['threat_level']:.2f} detected",
                'assigned_team': 'Security Operations Center',
                'sla': '1 hour' if report['executive_summary']['threat_category'] == 'CRITICAL' else '4 hours'
            }
        })
    
    # 2. Based on ML predictions
    if 'performance' in ml_results and ml_results['performance']['roc_auc'] > 0.8:
        operations_actions.append({
            'type': 'ml_enhancement',
            'priority': 'MEDIUM',
            'action': 'UPDATE_ML_MODELS',
            'details': {
                'model_performance': f"ROC AUC: {ml_results['performance']['roc_auc']:.3f}",
                'update_type': 'Production deployment',
                'validation_required': True
            }
        })
    
    # 3. Based on threat hunting
    high_priority_hunts = [h for h in hunting_results if h['priority'] == 'HIGH']
    if high_priority_hunts:
        operations_actions.append({
            'type': 'threat_hunting',
            'priority': 'HIGH',
            'action': 'INVESTIGATE_HUNTING_FINDINGS',
            'details': {
                'findings_count': len(high_priority_hunts),
                'scenarios': [h['scenario'] for h in high_priority_hunts],
                'investigation_team': 'Threat Intelligence Team'
            }
        })
    
    # 4. General security operations
    operations_actions.extend([
        {
            'type': 'monitoring',
            'priority': 'MEDIUM',
            'action': 'UPDATE_MONITORING_RULES',
            'details': {
                'rule_updates': 'Add new threat patterns to monitoring',
                'implementation': 'Automated rule deployment',
                'testing_required': True
            }
        },
        {
            'type': 'reporting',
            'priority': 'LOW',
            'action': 'GENERATE_SECURITY_REPORT',
            'details': {
                'report_type': 'Weekly security summary',
                'audience': 'Security management',
                'delivery': 'Automated email and dashboard'
            }
        }
    ])
    
    # Create operations dashboard
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Operations actions by priority
    ax1 = axes[0, 0]
    priorities = ['HIGH', 'MEDIUM', 'LOW']
    action_counts = {p: 0 for p in priorities}
    
    for action in operations_actions:
        action_counts[action['priority']] += 1
    
    colors = ['red', 'orange', 'green']
    bars = ax1.bar(priorities, [action_counts[p] for p in priorities], color=colors, alpha=0.7)
    
    ax1.set_xlabel('Priority')
    ax1.set_ylabel('Number of Actions')
    ax1.set_title('Security Operations Actions by Priority')
    ax1.grid(True, alpha=0.3, axis='y')
    
    # Add value labels
    for bar, count in zip(bars, [action_counts[p] for p in priorities]):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{count}', ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    # 2. Action types distribution
    ax2 = axes[0, 1]
    action_types = list(set([a['type'] for a in operations_actions]))
    type_counts = {t: 0 for t in action_types}
    
    for action in operations_actions:
        type_counts[action['type']] += 1
    
    wedges, texts, autotexts = ax2.pie(
        [type_counts[t] for t in action_types],
        labels=action_types,
        autopct='%1.1f%%',
        startangle=90,
        colors=plt.cm.Set3(np.linspace(0, 1, len(action_types)))
    )
    
    ax2.set_title('Operations Actions by Type')
    
    # Enhance autotexts
    for autotext in autotexts:
        autotext.set_color('black')
        autotext.set_fontweight('bold')
    
    # 3. Operations timeline
    ax3 = axes[1, 0]
    
    # Simulated operation durations
    operations = [
        ('Threat Analysis', 15.2),
        ('ML Prediction', 3.5),
        ('Threat Hunting', 25.8),
        ('Response Planning', 8.3),
        ('Implementation', 12.7)
    ]
    
    operations_names = [op[0] for op in operations]
    durations = [op[1] for op in operations]
    
    y_pos = np.arange(len(operations))
    
    bars = ax3.barh(y_pos, durations, color=plt.cm.viridis(np.linspace(0, 1, len(operations))))
    ax3.set_yticks(y_pos)
    ax3.set_yticklabels(operations_names)
    ax3.set_xlabel('Duration (seconds)')
    ax3.set_title('Security Operations Timeline')
    ax3.grid(True, alpha=0.3, axis='x')
    
    # Add duration labels
    for bar, duration in zip(bars, durations):
        width = bar.get_width()
        ax3.text(width + 0.5, bar.get_y() + bar.get_height()/2,
                f'{duration:.1f}s', ha='left', va='center', fontsize=9)
    
    # 4. Integration status
    ax4 = axes[1, 1]
    
    status_items = list(integrated_data['operations_status'].keys())
    status_values = list(integrated_data['operations_status'].values())
    
    # Create gauge-style visualization
    theta = np.linspace(0, np.pi, len(status_items))
    r = np.ones_like(theta) * 1.2
    
    # Plot status indicators
    for i, (item, value) in enumerate(zip(status_items, status_values)):
        angle = theta[i]
        
        # Plot status indicator
        if value:
            color = 'green'
            marker = 'o'
            size = 100
            label = 'Active'
        else:
            color = 'red'
            marker = 'x'
            size = 100
            label = 'Inactive'
        
        ax4.scatter(angle, r[i], color=color, marker=marker, s=size, alpha=0.7)
        
        # Add text label
        text_angle = angle * 180 / np.pi
        if text_angle < 90 or text_angle > 270:
            ha = 'left'
        else:
            ha = 'right'
        
        ax4.text(angle, r[i] + 0.1, item.replace('_', '\n'), 
                ha=ha, va='center', fontsize=8, rotation=text_angle)
    
    ax4.set_xlim(-0.5, np.pi + 0.5)
    ax4.set_ylim(0, 2)
    ax4.set_aspect('equal')
    ax4.axis('off')
    ax4.set_title('Security Operations Status')
    
    plt.suptitle('Security Operations Integration Dashboard', 
                fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Print operations summary
    print(f"\n📋 OPERATIONS SUMMARY:")
    print(f"  Total actions generated: {len(operations_actions)}")
    print(f"  High priority actions: {action_counts['HIGH']}")
    print(f"  Medium priority actions: {action_counts['MEDIUM']}")
    print(f"  Low priority actions: {action_counts['LOW']}")
    
    print(f"\n🛡️ SECURITY STATUS:")
    for item, value in integrated_data['operations_status'].items():
        status = "✅ ACTIVE" if value else "❌ INACTIVE"
        print(f"  {item.replace('_', ' ').title()}: {status}")
    
    print(f"\n🎯 RECOMMENDED OPERATIONS:")
    for action in operations_actions:
        if action['priority'] == 'HIGH':
            print(f"  🚨 {action['action']} - {action['type'].replace('_', ' ').title()}")
    
    return {
        'integrated_data': integrated_data,
        'operations_actions': operations_actions,
        'summary': {
            'total_actions': len(operations_actions),
            'high_priority_actions': action_counts['HIGH'],
            'operations_status': integrated_data['operations_status']
        }
    }

# Integrate with security operations
print("\n🛡️ Integrating with Security Operations...")
operations_integration = security_operations_integration(report, ml_results, hunting_results)

# %% [markdown]
# ## 16. Conclusion and Next Steps

# %%
print("\n" + "="*80)
print("CYBERGUARD THREAT ANALYSIS - COMPLETE")
print("="*80)

print("\n✅ ANALYSIS COMPLETED SUCCESSFULLY")
print(f"📅 Analysis Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

if 'report' in locals():
    print(f"\n📊 KEY FINDINGS FOR: {report['metadata']['url']}")
    print(f"   • Threat Level: {report['executive_summary']['threat_level']:.2f} "
          f"({report['executive_summary']['threat_category']})")
    print(f"   • Risk Score: {report['executive_summary']['risk_score']:.2f}")
    print(f"   • Confidence: {report['executive_summary']['confidence']:.2f}")
    print(f"   • Vulnerabilities Found: {len(report['vulnerabilities'])}")
    print(f"   • Recommended Action: {report['executive_summary']['recommended_action']}")

if 'batch_df' in locals() and not batch_df.empty:
    print(f"\n📈 BATCH ANALYSIS SUMMARY:")
    print(f"   • Websites Analyzed: {len(batch_df)}")
    print(f"   • Average Threat Level: {batch_df['threat_level'].mean():.2f}")
    print(f"   • Critical Vulnerabilities: {batch_df['critical_vulns'].sum()}")

if 'ml_results' in locals():
    print(f"\n🤖 MACHINE LEARNING PERFORMANCE:")
    print(f"   • Threat Prediction R²: {ml_results['performance']['threat_r2']:.3f}")
    print(f"   • Attack Prediction AUC: {ml_results['performance']['roc_auc']:.3f}")

if 'operations_integration' in locals():
    print(f"\n🛡️ SECURITY OPERATIONS:")
    print(f"   • Total Operations Actions: {operations_integration['summary']['total_actions']}")
    print(f"   • High Priority Actions: {operations_integration['summary']['high_priority_actions']}")

print(f"\n🔒 SECURITY RECOMMENDATIONS:")
print("   1. Implement all critical and high-priority recommendations immediately")
print("   2. Schedule security review for medium-priority findings")
print("   3. Update monitoring rules based on threat intelligence")
print("   4. Conduct regular proactive threat hunting")
print("   5. Continuously train and update ML models")

print(f"\n📅 NEXT STEPS:")
print("   1. Review automated responses and implement immediately")
print("   2. Schedule follow-up scans for critical findings")
print("   3. Update security policies based on findings")
print("   4. Conduct security awareness training for relevant teams")
print("   5. Schedule next comprehensive security assessment")

print(f"\n📞 CONTACT:")
print("   • Security Operations: soc@company.com")
print("   • Incident Response: irt@company.com")
print("   • Threat Intelligence: threatintel@company.com")

print(f"\n" + "="*80)
print("Thank you for using CyberGuard Threat Analysis System!")
print("="*80)

# Save final notebook state
print("\n💾 Saving notebook state...")

# Create summary markdown cell
summary_markdown = f"""
## 🎉 Analysis Complete - Summary

**Analysis Completed**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

### Key Metrics:
- **Websites Analyzed**: {len(batch_df) if 'batch_df' in locals() and not batch_df.empty else 1}
- **Average Threat Level**: {batch_df['threat_level'].mean() if 'batch_df' in locals() and not batch_df.empty else report['executive_summary']['threat_level']:.2f}
- **Total Vulnerabilities**: {len(report['vulnerabilities']) if 'report' in locals() else 0}
- **ML Prediction Accuracy**: {ml_results['performance']['attack_accuracy']:.3f if 'ml_results' in locals() else 'N/A'}

### Next Actions:
1. Review automated response recommendations
2. Implement critical security controls
3. Schedule follow-up assessments
4. Update security monitoring rules

### Files Generated:
- Security reports in JSON/HTML format
- Batch analysis CSV
- Threat intelligence correlations
- Automated response plans
- Machine learning models

### Notes:
This analysis was conducted using CyberGuard's advanced threat detection capabilities including multi-agent coordination, machine learning prediction, and real-time threat intelligence integration.
"""

print(summary_markdown)